In [41]:
import xml.etree.ElementTree as Xet
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import warnings
import re

# Ignore all warnings
warnings.filterwarnings('ignore')

# Kansas 2022 General Election Precinct Boundary Election Files

### Load Data

pber from IKEA Lab: https://www.ike-lab.com/electionresults.html

In [42]:
kansas_2022 = './ks_2022g (1)/ks_2022g.shp'
gdf_kansan_2022 = gpd.read_file(kansas_2022)
gdf_kansan_2022.head(5)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,...,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry
0,001,000010,Carlyle Township Voting District,28,165,42,0,235,104,19,...,53,66,56,67,55,65,57,72,48,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42..."
1,001,000020,Cottage Grove Township Voting District,19,108,43,1,171,72,16,...,48,29,49,36,42,33,45,44,34,"POLYGON Z ((-95.46656 37.75217 0.00000, -95.46..."
2,001,000030,Deer Creek Township Voting District,11,65,17,1,94,44,7,...,16,28,19,29,18,30,17,31,15,"POLYGON Z ((-95.33574 37.98185 0.00000, -95.33..."
3,001,00006A,Humboldt Township Voting District,29,127,53,2,211,104,15,...,42,60,43,59,43,63,39,61,38,"POLYGON Z ((-95.47322 37.84735 0.00000, -95.47..."
4,001,170040,Iola Ward 4,169,366,303,8,846,210,119,...,93,228,94,232,90,226,93,224,93,"POLYGON Z ((-95.40017 37.90415 0.00000, -95.40..."


In [43]:
len(gdf_kansan_2022)

4379

In [44]:
gdf_kansan_2022.columns

Index(['COUNTYFP', 'VTDST', 'NAMELSAD', 'REG_DEM', 'REG_REP', 'REG_UNA',
       'REG_LIB', 'REG_TOTAL', 'G22USSRMOR', 'G22USSDHOL', 'G22USSLGRA',
       'G22GOVDKEL', 'G22GOVRSCH', 'G22GOVIPYL', 'G22GOVLCOR', 'G22SOSRSCH',
       'G22SOSDREP', 'G22SOSLLAN', 'G22ATGRKOB', 'G22ATGDMAN', 'G22TRERJOH',
       'G22TREDROG', 'G22TRELROB', 'G22INSRSCH', 'G22INSDCOR', 'G22SSC1YES',
       'G22SSC1NO', 'G22SSC2YES', 'G22SSC2NO', 'G22SSC3YES', 'G22SSC3NO',
       'G22SSC5YES', 'G22SSC5NO', 'G22SSC6YES', 'G22SSC6NO', 'G22SSC7YES',
       'G22SSC7NO', 'geometry'],
      dtype='object')

What's already there: US Senate, Governor/Lieutenant Governor, Secretary of State, Attorney General, State Treasurer, Insurance Commissioner, Kansas Supreme Court Justices

What's missing: Kansas senate, Kansas House of representatives, Constitutional Amendment Rules and Regulations, Sheriff Election and Recall

#### Kansas Senate: state senates are elected at year 2020, 2024..., not in 2022 general election, except for Senate 38

In [45]:
senate_csv_path = './2022-General-Election-Kansas-Senate-Results-By-Precinct.xlsx'

all_sheets = pd.read_excel(senate_csv_path, sheet_name=None)

print("Sheet names:", all_sheets.keys())

dfs_senate = {}  

for sheet_name, df in all_sheets.items():
    print(f"Processing sheet: {sheet_name}")
    dfs_senate[sheet_name] = df 

Sheet names: dict_keys(['KS Senate'])
Processing sheet: KS Senate


Only Kansas Senate 38 is being elected

In [46]:
KS_Senate = dfs_senate['KS Senate']

# Function to remove middle name, swap first and last names, and remove "Jr." or "Sr."
def remove_middle_swap_and_suffix(name):
    try:
        parts = name.split()
        if len(parts) == 2:  # Only first and last name
            return f"{parts[1]} {parts[0]}"  # Swap first and last
        elif len(parts) > 2:  # More than two parts
            # Handle cases with suffix (e.g., "John Doe Jr.")
            if parts[-1].lower() in ["jr", "sr"]:  # Remove suffix
                return f"{parts[-2]} {parts[0]}"  # Swap first and last, remove suffix
            else:
                return f"{parts[-1]} {parts[0]}"  # Swap first and last, remove middle name
        else:
            return name  # In case there's only one part (single name)
    except:
        return name

# Apply the function to the 'Candidate' column
KS_Senate['Candidate'] = KS_Senate['Candidate'].apply(remove_middle_swap_and_suffix)


KS_Senate.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Clark,Appleton Township,Kansas Senate 38,"Jose Lara,",Democratic,30,000010
1,Clark,Appleton Township,Kansas Senate 38,W. Ryckman,Republican,282,000010
2,Clark,Center 1,Kansas Senate 38,"Jose Lara,",Democratic,27,000020
3,Clark,Center 1,Kansas Senate 38,W. Ryckman,Republican,189,000020
4,Clark,Center 2,Kansas Senate 38,"Jose Lara,",Democratic,18,000030


In [47]:
election_results_cleaned = KS_Senate

# Extract district number from the 'Race' column and create new column
election_results_cleaned['District'] = election_results_cleaned['Race'].str.extract(r'(\d+)', expand=False)

# Function to create a unique column name based on the office, district, candidate, and party
def create_column_name(row):
    office_abbr = "SU"  # Abbreviation for State Legislature
    party_initial = row['Party'][0]
    last_name = row['Candidate'].split()[-1][:3].upper()
    district = row['District'].zfill(3)  # Ensure the district number has leading zeros
    
    # Create a unique column name that includes office, district, candidate, and party
    unique_name = f"{office_abbr}{district}{party_initial}{last_name}"
    
    return unique_name

# Apply the function to generate a unique column name for each candidate
election_results_cleaned['ColumnName'] = election_results_cleaned.apply(create_column_name, axis=1)

# Pivot dataset so that each column represents a candidate (unique column name) and each row represents a precinct
pivoted_df_senate = election_results_cleaned.pivot_table(
    index=['County', 'Precinct', 'VTD'],
    columns='ColumnName',
    values='Votes',
    aggfunc='sum'
)

# Flatten the columns
pivoted_df_senate.columns = [col for col in pivoted_df_senate.columns]

# Reset index
pivoted_df_senate.reset_index(inplace=True)

# Display the final DataFrame
pivoted_df_senate.head(5)

,County,Precinct,VTD,SU038DLAR,SU038RRYC
0,Clark,Appleton Township,000010,30,282
1,Clark,Center 1,000020,27,189
2,Clark,Center 2,000030,18,165
3,Clark,Englewood Township,000040,14,50
4,Clark,Lexington Township,000050,0,21


In [48]:
pd.set_option('display.max_rows', None)
pivoted_df_senate

,County,Precinct,VTD,SU038DLAR,SU038RRYC
0,Clark,Appleton Township,000010,30,282
1,Clark,Center 1,000020,27,189
2,Clark,Center 2,000030,18,165
3,Clark,Englewood Township,000040,14,50
4,Clark,Lexington Township,000050,0,21
5,Clark,Liberty Township,000060,0,11
6,Clark,Sitka Township,000070,3,27
7,Ford,Bloom Township,000010,0,27
8,Ford,Bucklin City,000020,27,224
9,Ford,Bucklin Township,000030,3,42


#### Notice that while the election result has a county name column, the pber file does not, I add this county fips/names matching file for better matching

In [49]:
county_fips_path = './US_FIPS_Codes (1).xls'

county_fip = pd.read_excel(county_fips_path)
kansas_fip = county_fip[county_fip['U.S. FIPS County Codes'] == 'Kansas']

kansas_fip.drop(columns=['U.S. FIPS County Codes', 'Unnamed: 2'], inplace=True)
kansas_fip.rename(columns={'Unnamed: 3': 'CountyFP'}, inplace=True)
kansas_fip.rename(columns={'Unnamed: 1': 'COUNTYNA'}, inplace=True)
kansas_fip.head()


,COUNTYNA,CountyFP
886,Allen,001
887,Anderson,003
888,Atchison,005
889,Barber,007
890,Barton,009


In [50]:
gdf_kansan_2022 = pd.merge(gdf_kansan_2022, kansas_fip, left_on='COUNTYFP', right_on='CountyFP', how='outer', indicator=True)
gdf_kansan_2022.head(5)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,...,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,CountyFP,_merge
0,001,000010,Carlyle Township Voting District,28,165,42,0,235,104,19,...,67,55,65,57,72,48,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42...",Allen,001,both
1,001,000020,Cottage Grove Township Voting District,19,108,43,1,171,72,16,...,36,42,33,45,44,34,"POLYGON Z ((-95.46656 37.75217 0.00000, -95.46...",Allen,001,both
2,001,000030,Deer Creek Township Voting District,11,65,17,1,94,44,7,...,29,18,30,17,31,15,"POLYGON Z ((-95.33574 37.98185 0.00000, -95.33...",Allen,001,both
3,001,00006A,Humboldt Township Voting District,29,127,53,2,211,104,15,...,59,43,63,39,61,38,"POLYGON Z ((-95.47322 37.84735 0.00000, -95.47...",Allen,001,both
4,001,170040,Iola Ward 4,169,366,303,8,846,210,119,...,232,90,226,93,224,93,"POLYGON Z ((-95.40017 37.90415 0.00000, -95.40...",Allen,001,both


Make sure all fips and names match

In [51]:
gdf_kansan_2022['_merge'].value_counts()

_merge
both          4379
left_only        0
right_only       0
Name: count, dtype: int64

Drop duplicated columns

In [52]:
gdf_kansan_2022 = gdf_kansan_2022.drop(columns=['CountyFP', '_merge'])

In [53]:
pber_county_precinct_counts = gdf_kansan_2022.groupby('COUNTYNA')['NAMELSAD'].nunique().reset_index()
pber_county_precinct_counts.columns = ['COUNTYNA', 'Unique Precinct Count']
pber_county_precinct_counts.head()


,COUNTYNA,Unique Precinct Count
0,Allen,26
1,Anderson,18
2,Atchison,27
3,Barber,22
4,Barton,51


# Kansas House

The election result is an excel file with 5 different sheet. The first sheet Kansas2022 has results for 101 counites, and 4 other sheets have results for the other 4 counties with a large number of precincts

In [54]:
house_csv_path = './2022-General-Election-Kansas-House-of-Representatives-Results-By-Precinct.xlsx'

all_sheets = pd.read_excel(house_csv_path, sheet_name=None)

print("Sheet names:", all_sheets.keys())

dfs = {}  

for sheet_name, df in all_sheets.items():
    dfs[sheet_name] = df 

    print(f"Processing sheet: {sheet_name}")



Sheet names: dict_keys(['KSHouse', 'JOHNSON', 'SEDGWICK', 'SHAWNEE', 'WYANDOTTE'])
Processing sheet: KSHouse
Processing sheet: JOHNSON
Processing sheet: SEDGWICK
Processing sheet: SHAWNEE
Processing sheet: WYANDOTTE


In [55]:
KSHouse = dfs['KSHouse']
JOHNSON_COUNTY = dfs['JOHNSON']
SEDGWICK_COUNTY = dfs['SEDGWICK']
SHAWNEE_COUNTY = dfs['SHAWNEE']
WYANDOTTE_COUNTY = dfs['WYANDOTTE']

### Process Sedgwick County

This county file does not have a VTD, use precinct code first

In [56]:
len(SEDGWICK_COUNTY)

366

In [57]:
candidate_party_info = SEDGWICK_COUNTY.iloc[0, 2:] 

df_with_party_cleaned = SEDGWICK_COUNTY.iloc[1:].reset_index(drop=True)

def split_party_candidate(column_name):
    parts = column_name.split(' ', 1)
    return parts[0], parts[1] if len(parts) > 1 else ''  # Split into party and candidate

results = []
for idx, row in df_with_party_cleaned.iterrows():
    precinct_code = row['PRECINCT CODE']
    precinct_name = row['PRECINCT NAME']
    
    for col in candidate_party_info.index:
        if row[col] >= 0:
            party, candidate = split_party_candidate(candidate_party_info[col])
            votes = row[col]
            race = col.split('.')[0] 
            
            results.append({
                'County': 'Sedgwick',
                'Precinct': precinct_name,
                'Race': race,
                'Candidate': candidate,
                'Party': party,
                'Votes': votes,
                'VTD': precinct_code
            })

results_with_party_df = pd.DataFrame(results)

results_with_party_df.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Sedgwick,101,State Representative 72nd District,Avery Anderson,REP,0,1
1,Sedgwick,101,State Representative 81st District,Blake Carpenter,REP,0,1
2,Sedgwick,101,State Representative 82nd District,Misti Hobbs,DEM,0,1
3,Sedgwick,101,State Representative 82nd District,Leah Howell,REP,0,1
4,Sedgwick,101,State Representative 83rd District,Henry Helgerson,DEM,0,1


In [58]:
pd.set_option('display.max_rows', None)
House_results_Sedgwick = results_with_party_df[['County', 'Precinct', 'Race', 'Candidate', 'Party', 'Votes', 'VTD']]
House_results_Sedgwick.head(5)

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Sedgwick,101,State Representative 72nd District,Avery Anderson,REP,0,1
1,Sedgwick,101,State Representative 81st District,Blake Carpenter,REP,0,1
2,Sedgwick,101,State Representative 82nd District,Misti Hobbs,DEM,0,1
3,Sedgwick,101,State Representative 82nd District,Leah Howell,REP,0,1
4,Sedgwick,101,State Representative 83rd District,Henry Helgerson,DEM,0,1


The file also contains county total rows. Separate them from the df and make it a new df for references

In [59]:
county_totals_df_Sedgwick = House_results_Sedgwick[House_results_Sedgwick['Precinct'] == 'COUNTY TOTALS']

House_results_Sedgwick = House_results_Sedgwick[House_results_Sedgwick['Precinct'] != 'COUNTY TOTALS']
House_results_Sedgwick.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Sedgwick,101,State Representative 72nd District,Avery Anderson,REP,0,1
1,Sedgwick,101,State Representative 81st District,Blake Carpenter,REP,0,1
2,Sedgwick,101,State Representative 82nd District,Misti Hobbs,DEM,0,1
3,Sedgwick,101,State Representative 82nd District,Leah Howell,REP,0,1
4,Sedgwick,101,State Representative 83rd District,Henry Helgerson,DEM,0,1


In [60]:
House_results_Sedgwick['Precinct'].nunique()

364

In [61]:
Sedgwick_PBER = gdf_kansan_2022[gdf_kansan_2022['COUNTYNA'] == 'Sedgwick']
len(Sedgwick_PBER)

518

### Process Johnson County

Check the format of 2 files

In [62]:
Johnson_PBER = gdf_kansan_2022[gdf_kansan_2022['COUNTYNA'] == 'Johnson']
Johnson_PBER.head()
Johnson_PBER['NAMELSAD'].unique()

array(['Aubry Twp 0-01', 'Aubry Twp 0-02', 'Aubry Twp 0-03',
       'Aubry Twp 0-04', 'Aubry Twp 0-05', 'Aubry Twp 0-06',
       'Aubry Twp 0-07', 'Aubry Twp 0-08', 'Bonner Springs 4-01',
       'De Soto 0-01', 'De Soto 0-02', 'De Soto 0-03', 'De Soto 0-04',
       'De Soto 0-05', 'De Soto 0-06', 'De Soto 0-07', 'De Soto 0-08',
       'De Soto 0-09', 'De Soto 0-10', 'De Soto 0-11', 'De Soto 0-12',
       'Edgerton 0-01', 'Edgerton 0-02', 'Edgerton 0-03', 'Edgerton 0-04',
       'Edgerton 0-05', 'Edgerton 0-06', 'Edgerton 0-07', 'Edgerton 0-08',
       'Edgerton 0-09', 'Edgerton 0-10', 'Fairway 1-01', 'Fairway 1-02',
       'Fairway 2-01', 'Fairway 2-02', 'Fairway 3-01', 'Fairway 3-02',
       'Fairway 4-01', 'Fairway 4-02', 'Fairway 4-03', 'Fairway 4-04',
       'Gardner 0-01', 'Gardner 0-02', 'Gardner 0-03', 'Gardner 0-04',
       'Gardner 0-05', 'Gardner 0-06', 'Gardner 0-07', 'Gardner 0-08',
       'Gardner 0-09', 'Gardner 0-10', 'Gardner 0-11', 'Gardner 0-12',
       'Gardner 0-13'

In [63]:
JOHNSON_COUNTY['PRECINCT NAME'].unique()

array([nan, 'Aubry Township Precinct 01', 'Aubry Township Precinct 02',
       'Aubry Township Precinct 03', 'Aubry Township Precinct 04',
       'Aubry Township Precinct 05', 'Aubry Township Precinct 06',
       'Aubry Township Precinct 07', 'Aubry Township Precinct 08',
       'Gardner Township Precinct 01', 'Gardner Township Precinct 02',
       'Gardner Township Precinct 03', 'Gardner Township Precinct 04',
       'Gardner Township Precinct 05', 'Gardner Township Precinct 06',
       'Gardner Township Precinct 07', 'Gardner Township Precinct 08',
       'Gardner Township Precinct 09', 'Gardner Township Precinct10',
       'Gardner Township Precinct11', 'Gardner Township Precinct12',
       'Gardner Township Precinct13', 'Gardner Township Precinct14',
       'Gardner Township Precinct15', 'Gardner Township Precinct16',
       'Gardner Township Precinct17', 'Gardner Township Precinct18',
       'Gardner Township Precinct19', 'Gardner Township Precinct20',
       'Gardner Township Pre

Since the 2 files have the same number of unique precincts, try to match 1 by 1 and do some replacement

In [64]:
list1 = ['Aubry Twp 0-01', 'Aubry Twp 0-02', 'Aubry Twp 0-03',
       'Aubry Twp 0-04', 'Aubry Twp 0-05', 'Aubry Twp 0-06',
       'Aubry Twp 0-07', 'Aubry Twp 0-08', 'Bonner Springs 4-01',
       'De Soto 0-01', 'De Soto 0-02', 'De Soto 0-03', 'De Soto 0-04',
       'De Soto 0-05', 'De Soto 0-06', 'De Soto 0-07', 'De Soto 0-08',
       'De Soto 0-09', 'De Soto 0-10', 'De Soto 0-11', 'De Soto 0-12',
       'Edgerton 0-01', 'Edgerton 0-02', 'Edgerton 0-03', 'Edgerton 0-04',
       'Edgerton 0-05', 'Edgerton 0-06', 'Edgerton 0-07', 'Edgerton 0-08',
       'Edgerton 0-09', 'Edgerton 0-10', 'Fairway 1-01', 'Fairway 1-02',
       'Fairway 2-01', 'Fairway 2-02', 'Fairway 3-01', 'Fairway 3-02',
       'Fairway 4-01', 'Fairway 4-02', 'Fairway 4-03', 'Fairway 4-04',
       'Gardner 0-01', 'Gardner 0-02', 'Gardner 0-03', 'Gardner 0-04',
       'Gardner 0-05', 'Gardner 0-06', 'Gardner 0-07', 'Gardner 0-08',
       'Gardner 0-09', 'Gardner 0-10', 'Gardner 0-11', 'Gardner 0-12',
       'Gardner 0-13', 'Gardner 0-14', 'Gardner 0-15', 'Gardner 0-16',
       'Gardner 0-17', 'Gardner 0-18', 'Gardner 0-19', 'Gardner 0-20',
       'Gardner 0-21', 'Gardner 0-22', 'Gardner Twp 0-01',
       'Gardner Twp 0-02', 'Gardner Twp 0-03', 'Gardner Twp 0-04',
       'Gardner Twp 0-05', 'Gardner Twp 0-06', 'Gardner Twp 0-07',
       'Gardner Twp 0-08', 'Gardner Twp 0-09', 'Gardner Twp 0-10',
       'Gardner Twp 0-11', 'Gardner Twp 0-12', 'Gardner Twp 0-13',
       'Gardner Twp 0-14', 'Gardner Twp 0-15', 'Gardner Twp 0-16',
       'Gardner Twp 0-17', 'Gardner Twp 0-18', 'Gardner Twp 0-19',
       'Gardner Twp 0-20', 'Gardner Twp 0-21', 'Gardner Twp 0-22',
       'Gardner Twp 0-23', 'Gardner Twp 0-24', 'Gardner Twp 0-25',
       'Gardner Twp 0-26', 'Gardner Twp 0-27', 'Lake Quivira 0-01',
       'Leawood 1-01', 'Leawood 1-02', 'Leawood 1-03', 'Leawood 1-04',
       'Leawood 1-05', 'Leawood 1-06', 'Leawood 1-07', 'Leawood 2-01',
       'Leawood 2-02', 'Leawood 2-03', 'Leawood 2-04', 'Leawood 2-05',
       'Leawood 2-06', 'Leawood 2-07', 'Leawood 3-01', 'Leawood 3-02',
       'Leawood 3-03', 'Leawood 3-04', 'Leawood 3-05', 'Leawood 3-06',
       'Leawood 3-07', 'Leawood 4-01', 'Leawood 4-02', 'Leawood 4-03',
       'Leawood 4-04', 'Leawood 4-05', 'Leawood 4-06', 'Leawood 4-07',
       'Lenexa 1-01', 'Lenexa 1-02', 'Lenexa 1-03', 'Lenexa 1-04',
       'Lenexa 1-05', 'Lenexa 1-06', 'Lenexa 1-07', 'Lenexa 1-08',
       'Lenexa 1-09', 'Lenexa 1-10', 'Lenexa 1-11', 'Lenexa 2-01',
       'Lenexa 2-02', 'Lenexa 2-03', 'Lenexa 2-04', 'Lenexa 2-05',
       'Lenexa 2-06', 'Lenexa 2-07', 'Lenexa 2-08', 'Lenexa 2-09',
       'Lenexa 2-10', 'Lenexa 2-11', 'Lenexa 3-01', 'Lenexa 3-02',
       'Lenexa 3-03', 'Lenexa 3-04', 'Lenexa 3-05', 'Lenexa 3-06',
       'Lenexa 3-07', 'Lenexa 3-08', 'Lenexa 3-09', 'Lenexa 3-10',
       'Lenexa 4-01', 'Lenexa 4-02', 'Lenexa 4-03', 'Lenexa 4-04',
       'Lenexa 4-05', 'Lenexa 4-06', 'Lenexa 4-07', 'Lenexa 4-08',
       'Lenexa 4-09', 'Lenexa 4-10', 'Lenexa 4-11', 'Lenexa 4-12',
       'Lenexa 4-13', 'Lenexa 4-14', 'Lenexa 4-15', 'Lexington Twp 0-01',
       'Lexington Twp 0-02', 'Lexington Twp 0-03', 'Lexington Twp 0-04',
       'Lexington Twp 0-05', 'Lexington Twp 0-06', 'Lexington Twp 0-07',
       'Lexington Twp 0-08', 'Lexington Twp 0-09', 'McCamish Twp 0-01',
       'McCamish Twp 0-02', 'McCamish Twp 0-03', 'McCamish Twp 0-04',
       'McCamish Twp 0-05', 'McCamish Twp 0-06', 'McCamish Twp 0-07',
       'Merriam 1-01', 'Merriam 1-02', 'Merriam 1-03', 'Merriam 1-04',
       'Merriam 1-05', 'Merriam 2-01', 'Merriam 2-02', 'Merriam 2-03',
       'Merriam 2-04', 'Merriam 3-01', 'Merriam 3-02', 'Merriam 4-01',
       'Merriam 4-02', 'Merriam 4-03', 'Mission 1-01', 'Mission 1-02',
       'Mission 2-01', 'Mission 2-02', 'Mission 3-01', 'Mission 3-02',
       'Mission 4-01', 'Mission 4-02', 'Mission 4-03', 'Mission 4-04',
       'Mission Hills 0-01', 'Mission Hills 0-02', 'Mission Hills 0-03',
       'Mission Hills 0-04', 'Mission Woods 0-01', 'Olathe 1-01',
       'Olathe 1-02', 'Olathe 1-03', 'Olathe 1-04', 'Olathe 1-05',
       'Olathe 1-06', 'Olathe 1-07', 'Olathe 1-08', 'Olathe 1-09',
       'Olathe 1-10', 'Olathe 1-11', 'Olathe 1-12', 'Olathe 1-13',
       'Olathe 1-14', 'Olathe 1-15', 'Olathe 1-16', 'Olathe 1-17',
       'Olathe 1-18', 'Olathe 1-19', 'Olathe 1-20', 'Olathe 1-21',
       'Olathe 1-22', 'Olathe 1-23', 'Olathe 1-24', 'Olathe 1-25',
       'Olathe 1-26', 'Olathe 1-27', 'Olathe 1-28', 'Olathe 1-29',
       'Olathe 1-30', 'Olathe 1-31', 'Olathe 1-32', 'Olathe 1-33',
       'Olathe 2-01', 'Olathe 2-02', 'Olathe 2-03', 'Olathe 2-04',
       'Olathe 2-05', 'Olathe 2-06', 'Olathe 2-07', 'Olathe 2-08',
       'Olathe 2-09', 'Olathe 2-10', 'Olathe 2-11', 'Olathe 2-12',
       'Olathe 2-13', 'Olathe 2-14', 'Olathe 2-15', 'Olathe 2-16',
       'Olathe 2-17', 'Olathe 2-18', 'Olathe 2-19', 'Olathe 2-20',
       'Olathe 2-21', 'Olathe 2-22', 'Olathe 2-23', 'Olathe 2-24',
       'Olathe 2-25', 'Olathe 2-26', 'Olathe 2-27', 'Olathe 3-01',
       'Olathe 3-02', 'Olathe 3-03', 'Olathe 3-04', 'Olathe 3-05',
       'Olathe 3-06', 'Olathe 3-07', 'Olathe 3-08', 'Olathe 3-09',
       'Olathe 3-10', 'Olathe 3-11', 'Olathe 3-12', 'Olathe 3-13',
       'Olathe 3-14', 'Olathe 3-15', 'Olathe 3-16', 'Olathe 3-17',
       'Olathe 3-18', 'Olathe 3-19', 'Olathe 3-20', 'Olathe 3-21',
       'Olathe 3-22', 'Olathe 3-23', 'Olathe 3-24', 'Olathe 3-25',
       'Olathe 3-26', 'Olathe 3-27', 'Olathe 3-28', 'Olathe 3-29',
       'Olathe 4-01', 'Olathe 4-02', 'Olathe 4-03', 'Olathe 4-04',
       'Olathe 4-05', 'Olathe 4-06', 'Olathe 4-07', 'Olathe 4-08',
       'Olathe 4-09', 'Olathe 4-10', 'Olathe 4-11', 'Olathe 4-12',
       'Olathe 4-13', 'Olathe 4-14', 'Olathe 4-15', 'Olathe 4-16',
       'Olathe 4-17', 'Olathe 4-18', 'Olathe 4-19', 'Olathe 4-20',
       'Olathe 4-21', 'Olathe 4-22', 'Olathe 4-23', 'Olathe 4-24',
       'Olathe Twp 0-01', 'Olathe Twp 0-02', 'Olathe Twp 0-03',
       'Olathe Twp 0-04', 'Olathe Twp 0-05', 'Olathe Twp 0-06',
       'Olathe Twp 0-07', 'Olathe Twp 0-08', 'Olathe Twp 0-09',
       'Olathe Twp 0-10', 'Olathe Twp 0-11', 'Olathe Twp 0-12',
       'Olathe Twp 0-13', 'Olathe Twp 0-14', 'Olathe Twp 0-15',
       'Olathe Twp 0-16', 'Olathe Twp 0-17', 'Olathe Twp 0-18',
       'Olathe Twp 0-19', 'Olathe Twp 0-24', 'Olathe Twp 0-25',
       'Olathe Twp 0-26', 'Olathe Twp 0-32', 'Overland Park 1-01',
       'Overland Park 1-02', 'Overland Park 1-03', 'Overland Park 1-04',
       'Overland Park 1-05', 'Overland Park 1-06', 'Overland Park 1-07',
       'Overland Park 1-08', 'Overland Park 1-09', 'Overland Park 1-10',
       'Overland Park 1-11', 'Overland Park 1-12', 'Overland Park 1-13',
       'Overland Park 1-14', 'Overland Park 1-15', 'Overland Park 1-16',
       'Overland Park 1-17', 'Overland Park 1-18', 'Overland Park 1-19',
       'Overland Park 1-20', 'Overland Park 1-21', 'Overland Park 1-22',
       'Overland Park 1-23', 'Overland Park 1-24', 'Overland Park 1-25',
       'Overland Park 1-26', 'Overland Park 1-27', 'Overland Park 2-01',
       'Overland Park 2-02', 'Overland Park 2-03', 'Overland Park 2-04',
       'Overland Park 2-05', 'Overland Park 2-06', 'Overland Park 2-07',
       'Overland Park 2-08', 'Overland Park 2-09', 'Overland Park 2-10',
       'Overland Park 2-11', 'Overland Park 2-12', 'Overland Park 2-13',
       'Overland Park 2-14', 'Overland Park 2-15', 'Overland Park 2-16',
       'Overland Park 2-17', 'Overland Park 2-18', 'Overland Park 2-19',
       'Overland Park 2-20', 'Overland Park 2-21', 'Overland Park 2-22',
       'Overland Park 2-23', 'Overland Park 2-24', 'Overland Park 2-25',
       'Overland Park 2-26', 'Overland Park 2-27', 'Overland Park 2-28',
       'Overland Park 2-29', 'Overland Park 2-30', 'Overland Park 2-31',
       'Overland Park 3-01', 'Overland Park 3-02', 'Overland Park 3-03',
       'Overland Park 3-04', 'Overland Park 3-05', 'Overland Park 3-06',
       'Overland Park 3-07', 'Overland Park 3-08', 'Overland Park 3-09',
       'Overland Park 3-10', 'Overland Park 3-11', 'Overland Park 3-12',
       'Overland Park 3-13', 'Overland Park 3-14', 'Overland Park 3-15',
       'Overland Park 3-16', 'Overland Park 3-17', 'Overland Park 3-18',
       'Overland Park 3-19', 'Overland Park 3-20', 'Overland Park 3-21',
       'Overland Park 3-22', 'Overland Park 3-23', 'Overland Park 3-24',
       'Overland Park 3-25', 'Overland Park 3-26', 'Overland Park 4-01',
       'Overland Park 4-02', 'Overland Park 4-03', 'Overland Park 4-04',
       'Overland Park 4-05', 'Overland Park 4-06', 'Overland Park 4-07',
       'Overland Park 4-08', 'Overland Park 4-09', 'Overland Park 4-10',
       'Overland Park 4-11', 'Overland Park 4-12', 'Overland Park 4-13',
       'Overland Park 4-14', 'Overland Park 4-15', 'Overland Park 4-16',
       'Overland Park 4-17', 'Overland Park 4-18', 'Overland Park 4-19',
       'Overland Park 4-20', 'Overland Park 4-21', 'Overland Park 4-22',
       'Overland Park 5-01', 'Overland Park 5-02', 'Overland Park 5-03',
       'Overland Park 5-04', 'Overland Park 5-05', 'Overland Park 5-06',
       'Overland Park 5-07', 'Overland Park 5-08', 'Overland Park 5-09',
       'Overland Park 5-10', 'Overland Park 5-11', 'Overland Park 5-12',
       'Overland Park 5-13', 'Overland Park 5-14', 'Overland Park 5-15',
       'Overland Park 5-16', 'Overland Park 5-17', 'Overland Park 5-18',
       'Overland Park 5-19', 'Overland Park 5-20', 'Overland Park 6-01',
       'Overland Park 6-02', 'Overland Park 6-03', 'Overland Park 6-04',
       'Overland Park 6-05', 'Overland Park 6-06', 'Overland Park 6-07',
       'Overland Park 6-08', 'Overland Park 6-09', 'Overland Park 6-10',
       'Overland Park 6-11', 'Overland Park 6-12', 'Overland Park 6-13',
       'Overland Park 6-14', 'Overland Park 6-15', 'Overland Park 6-16',
       'Overland Park 6-17', 'Overland Park 6-18', 'Overland Park 6-19',
       'Overland Park 6-20', 'Overland Park 6-21', 'Overland Park 6-22',
       'Overland Park 6-23', 'Overland Park 6-24', 'Overland Park 6-25',
       'Oxford Twp 0-01', 'Oxford Twp 0-02', 'Oxford Twp 0-03',
       'Oxford Twp 0-04', 'Oxford Twp 0-05', 'Oxford Twp 0-08',
       'Oxford Twp 0-09', 'Oxford Twp 0-10', 'Oxford Twp 0-11',
       'Prairie Village 1-01', 'Prairie Village 1-02',
       'Prairie Village 1-03', 'Prairie Village 1-04',
       'Prairie Village 2-01', 'Prairie Village 2-02',
       'Prairie Village 2-03', 'Prairie Village 3-01',
       'Prairie Village 3-02', 'Prairie Village 3-03',
       'Prairie Village 4-01', 'Prairie Village 4-02',
       'Prairie Village 4-03', 'Prairie Village 5-01',
       'Prairie Village 5-02', 'Prairie Village 5-03',
       'Prairie Village 6-01', 'Prairie Village 6-02',
       'Prairie Village 6-03', 'Roeland Park 1-01', 'Roeland Park 1-02',
       'Roeland Park 2-01', 'Roeland Park 2-02', 'Roeland Park 3-01',
       'Roeland Park 3-02', 'Roeland Park 3-03', 'Roeland Park 4-01',
       'Roeland Park 4-02', 'Roeland Park 4-03', 'Shawnee 1-01',
       'Shawnee 1-02', 'Shawnee 1-03', 'Shawnee 1-04', 'Shawnee 1-05',
       'Shawnee 1-06', 'Shawnee 1-07', 'Shawnee 1-08', 'Shawnee 1-09',
       'Shawnee 1-10', 'Shawnee 1-11', 'Shawnee 1-12', 'Shawnee 1-13',
       'Shawnee 1-14', 'Shawnee 2-01', 'Shawnee 2-02', 'Shawnee 2-03',
       'Shawnee 2-04', 'Shawnee 2-05', 'Shawnee 2-06', 'Shawnee 2-07',
       'Shawnee 2-08', 'Shawnee 2-09', 'Shawnee 2-10', 'Shawnee 2-11',
       'Shawnee 2-12', 'Shawnee 2-13', 'Shawnee 3-01', 'Shawnee 3-02',
       'Shawnee 3-03', 'Shawnee 3-04', 'Shawnee 3-05', 'Shawnee 3-06',
       'Shawnee 3-07', 'Shawnee 3-08', 'Shawnee 3-09', 'Shawnee 3-10',
       'Shawnee 3-11', 'Shawnee 3-12', 'Shawnee 4-01', 'Shawnee 4-02',
       'Shawnee 4-03', 'Shawnee 4-04', 'Shawnee 4-05', 'Shawnee 4-06',
       'Shawnee 4-07', 'Shawnee 4-08', 'Shawnee 4-09', 'Shawnee 4-10',
       'Shawnee 4-11', 'Shawnee 4-12', 'Shawnee 4-13', 'Shawnee 4-14',
       'Shawnee 4-15', 'Spring Hill 0-01', 'Spring Hill 0-02',
       'Spring Hill 0-03', 'Spring Hill 0-04', 'Spring Hill 0-05',
       'Spring Hill 0-06', 'Spring Hill 0-07', 'Spring Hill Twp 0-01',
       'Spring Hill Twp 0-02', 'Spring Hill Twp 0-03',
       'Spring Hill Twp 0-04', 'Spring Hill Twp 0-05',
       'Spring Hill Twp 0-06', 'Spring Hill Twp 0-07',
       'Spring Hill Twp 0-08', 'Spring Hill Twp 0-09',
       'Spring Hill Twp 0-10', 'Westwood 0-01', 'Westwood 0-02',
       'Westwood Hills 0-01']

## For better matching after sorting, add the word 'ward' to Olathe, besides those Olathe Twp
list1 = [
    precinct.replace('Olathe', 'Olathe Ward') if precinct.startswith('Olathe ') and 'Twp' not in precinct else precinct
    for precinct in list1
]

list2 = ['Aubry Township Precinct 01', 'Aubry Township Precinct 02',
       'Aubry Township Precinct 03', 'Aubry Township Precinct 04',
       'Aubry Township Precinct 05', 'Aubry Township Precinct 06',
       'Aubry Township Precinct 07', 'Aubry Township Precinct 08',
       'Gardner Township Precinct 01', 'Gardner Township Precinct 02',
       'Gardner Township Precinct 03', 'Gardner Township Precinct 04',
       'Gardner Township Precinct 05', 'Gardner Township Precinct 06',
       'Gardner Township Precinct 07', 'Gardner Township Precinct 08',
       'Gardner Township Precinct 09', 'Gardner Township Precinct10',
       'Gardner Township Precinct11', 'Gardner Township Precinct12',
       'Gardner Township Precinct13', 'Gardner Township Precinct14',
       'Gardner Township Precinct15', 'Gardner Township Precinct16',
       'Gardner Township Precinct17', 'Gardner Township Precinct18',
       'Gardner Township Precinct19', 'Gardner Township Precinct20',
       'Gardner Township Precinct21', 'Gardner Township Precinct22',
       'Gardner Township Precinct23', 'Gardner Township Precinct24',
       'Gardner Township Precinct25', 'Gardner Township Precinct26',
       'Gardner Township Precinct27', 'Lexington Township Precinct 01',
       'Lexington Township Precinct 02', 'Lexington Township Precinct 03',
       'Lexington Township Precinct 04', 'Lexington Township Precinct 05',
       'Lexington Township Precinct 06', 'Lexington Township Precinct 07',
       'Lexington Township Precinct 08', 'Lexington Township Precinct 09',
       'McCamish Township Precinct 01', 'McCamish Township Precinct 02',
       'McCamish Township Precinct 03', 'McCamish Township Precinct 04',
       'McCamish Township Precinct 05', 'McCamish Township Precinct 06',
       'McCamish Township Precinct 07', 'Olathe Township Precinct 01',
       'Olathe Township Precinct 02', 'Olathe Township Precinct 03',
       'Olathe Township Precinct 04', 'Olathe Township Precinct 05',
       'Olathe Township Precinct 06', 'Olathe Township Precinct 07',
       'Olathe Township Precinct 08', 'Olathe Township Precinct 09',
       'Olathe Township Precinct10', 'Olathe Township Precinct11',
       'Olathe Township Precinct12', 'Olathe Township Precinct13',
       'Olathe Township Precinct14', 'Olathe Township Precinct15',
       'Olathe Township Precinct16', 'Olathe Township Precinct17',
       'Olathe Township Precinct18', 'Olathe Township Precinct19',
       'Olathe Township Precinct24', 'Olathe Township Precinct25',
       'Olathe Township Precinct26', 'Olathe Township Precinct32',
       'Oxford Township Precinct 01', 'Oxford Township Precinct 02',
       'Oxford Township Precinct 03', 'Oxford Township Precinct 04',
       'Oxford Township Precinct 05', 'Oxford Township Precinct 08',
       'Oxford Township Precinct 09', 'Oxford Township Precinct10',
       'Oxford Township Precinct11', 'Spring Hill Township Precinct 01',
       'Spring Hill Township Precinct 02',
       'Spring Hill Township Precinct 03',
       'Spring Hill Township Precinct 04',
       'Spring Hill Township Precinct 05',
       'Spring Hill Township Precinct 06',
       'Spring Hill Township Precinct 07',
       'Spring Hill Township Precinct 08',
       'Spring Hill Township Precinct 09',
       'Spring Hill Township Precinct10', 'De Soto Precinct 01',
       'De Soto Precinct 02', 'De Soto Precinct 03',
       'De Soto Precinct 04', 'De Soto Precinct 05',
       'De Soto Precinct 06', 'De Soto Precinct 07',
       'De Soto Precinct 08', 'De Soto Precinct 09',
       'De Soto Precinct 10', 'De Soto Precinct 11',
       'De Soto Precinct 12', 'Edgerton Precinct 01',
       'Edgerton Precinct 02', 'Edgerton Precinct 03',
       'Edgerton Precinct 04', 'Edgerton Precinct 05',
       'Edgerton Precinct 06', 'Edgerton Precinct 07',
       'Edgerton Precinct 08', 'Edgerton Precinct 09',
       'Edgerton Precinct 10', 'Gardner Precinct 01',
       'Gardner Precinct 02', 'Gardner Precinct 03',
       'Gardner Precinct 04', 'Gardner Precinct 05',
       'Gardner Precinct 06', 'Gardner Precinct 07',
       'Gardner Precinct 08', 'Gardner Precinct 09',
       'Gardner Precinct 10', 'Gardner Precinct 11',
       'Gardner Precinct 12', 'Gardner Precinct 13',
       'Gardner Precinct 14', 'Gardner Precinct 15',
       'Gardner Precinct 16', 'Gardner Precinct 17',
       'Gardner Precinct 18', 'Gardner Precinct 19',
       'Gardner Precinct 20', 'Gardner Precinct 21',
       'Gardner Precinct 22', 'Olathe Ward 1 Precinct 01',
       'Olathe Ward 1 Precinct 02', 'Olathe Ward 1 Precinct 03',
       'Olathe Ward 1 Precinct 04', 'Olathe Ward 1 Precinct 05',
       'Olathe Ward 1 Precinct 06', 'Olathe Ward 1 Precinct 07',
       'Olathe Ward 1 Precinct 08', 'Olathe Ward 1 Precinct 09',
       'Olathe Ward 1 Precinct 10', 'Olathe Ward 1 Precinct 11',
       'Olathe Ward 1 Precinct 12', 'Olathe Ward 1 Precinct 13',
       'Olathe Ward 1 Precinct 14', 'Olathe Ward 1 Precinct 15',
       'Olathe Ward 1 Precinct 16', 'Olathe Ward 1 Precinct 17',
       'Olathe Ward 1 Precinct 18', 'Olathe Ward 1 Precinct 19',
       'Olathe Ward 1 Precinct 20', 'Olathe Ward 1 Precinct 21',
       'Olathe Ward 1 Precinct 22', 'Olathe Ward 1 Precinct 23',
       'Olathe Ward 1 Precinct 24', 'Olathe Ward 1 Precinct 25',
       'Olathe Ward 1 Precinct 26', 'Olathe Ward 1 Precinct 27',
       'Olathe Ward 1 Precinct 28', 'Olathe Ward 1 Precinct 29',
       'Olathe Ward 1 Precinct 30', 'Olathe Ward 1 Precinct 31',
       'Olathe Ward 1 Precinct 32', 'Olathe Ward 1 Precinct 33',
       'Olathe Ward 2 Precinct 01', 'Olathe Ward 2 Precinct 02',
       'Olathe Ward 2 Precinct 03', 'Olathe Ward 2 Precinct 04',
       'Olathe Ward 2 Precinct 05', 'Olathe Ward 2 Precinct 06',
       'Olathe Ward 2 Precinct 07', 'Olathe Ward 2 Precinct 08',
       'Olathe Ward 2 Precinct 09', 'Olathe Ward 2 Precinct 10',
       'Olathe Ward 2 Precinct 11', 'Olathe Ward 2 Precinct 12',
       'Olathe Ward 2 Precinct 13', 'Olathe Ward 2 Precinct 14',
       'Olathe Ward 2 Precinct 15', 'Olathe Ward 2 Precinct 16',
       'Olathe Ward 2 Precinct 17', 'Olathe Ward 2 Precinct 18',
       'Olathe Ward 2 Precinct 19', 'Olathe Ward 2 Precinct 20',
       'Olathe Ward 2 Precinct 21', 'Olathe Ward 2 Precinct 22',
       'Olathe Ward 2 Precinct 23', 'Olathe Ward 2 Precinct 24',
       'Olathe Ward 2 Precinct 25', 'Olathe Ward 2 Precinct 26',
       'Olathe Ward 2 Precinct 27', 'Olathe Ward 3 Precinct 01',
       'Olathe Ward 3 Precinct 02', 'Olathe Ward 3 Precinct 03',
       'Olathe Ward 3 Precinct 04', 'Olathe Ward 3 Precinct 05',
       'Olathe Ward 3 Precinct 06', 'Olathe Ward 3 Precinct 07',
       'Olathe Ward 3 Precinct 08', 'Olathe Ward 3 Precinct 09',
       'Olathe Ward 3 Precinct 10', 'Olathe Ward 3 Precinct 11',
       'Olathe Ward 3 Precinct 12', 'Olathe Ward 3 Precinct 13',
       'Olathe Ward 3 Precinct 14', 'Olathe Ward 3 Precinct 15',
       'Olathe Ward 3 Precinct 16', 'Olathe Ward 3 Precinct 17',
       'Olathe Ward 3 Precinct 18', 'Olathe Ward 3 Precinct 19',
       'Olathe Ward 3 Precinct 20', 'Olathe Ward 3 Precinct 21',
       'Olathe Ward 3 Precinct 22', 'Olathe Ward 3 Precinct 23',
       'Olathe Ward 3 Precinct 24', 'Olathe Ward 3 Precinct 25',
       'Olathe Ward 3 Precinct 26', 'Olathe Ward 3 Precinct 27',
       'Olathe Ward 3 Precinct 28', 'Olathe Ward 3 Precinct 29',
       'Olathe Ward 4 Precinct 01', 'Olathe Ward 4 Precinct 02',
       'Olathe Ward 4 Precinct 03', 'Olathe Ward 4 Precinct 04',
       'Olathe Ward 4 Precinct 05', 'Olathe Ward 4 Precinct 06',
       'Olathe Ward 4 Precinct 07', 'Olathe Ward 4 Precinct 08',
       'Olathe Ward 4 Precinct 09', 'Olathe Ward 4 Precinct 10',
       'Olathe Ward 4 Precinct 11', 'Olathe Ward 4 Precinct 12',
       'Olathe Ward 4 Precinct 13', 'Olathe Ward 4 Precinct 14',
       'Olathe Ward 4 Precinct 15', 'Olathe Ward 4 Precinct 16',
       'Olathe Ward 4 Precinct 17', 'Olathe Ward 4 Precinct 18',
       'Olathe Ward 4 Precinct 19', 'Olathe Ward 4 Precinct 20',
       'Olathe Ward 4 Precinct 21', 'Olathe Ward 4 Precinct 22',
       'Olathe Ward 4 Precinct 23', 'Olathe Ward 4 Precinct 24',
       'Spring Hill Precinct 01', 'Spring Hill Precinct 02',
       'Spring Hill Precinct 03', 'Spring Hill Precinct 04',
       'Spring Hill Precinct 05', 'Spring Hill Precinct 06',
       'Spring Hill Precinct 07', 'Fairway Ward 1 Precinct 01',
       'Fairway Ward 1 Precinct 02', 'Fairway Ward 2 Precinct 01',
       'Fairway Ward 2 Precinct 02', 'Fairway Ward 3 Precinct 01',
       'Fairway Ward 3 Precinct 02', 'Fairway Ward 4 Precinct 01',
       'Fairway Ward 4 Precinct 02', 'Fairway Ward 4 Precinct 03',
       'Fairway Ward 4 Precinct 04', 'Leawood Ward 1 Precinct 01',
       'Leawood Ward 1 Precinct 02', 'Leawood Ward 1 Precinct 03',
       'Leawood Ward 1 Precinct 04', 'Leawood Ward 1 Precinct 05',
       'Leawood Ward 1 Precinct 06', 'Leawood Ward 1 Precinct 07',
       'Leawood Ward 2 Precinct 01', 'Leawood Ward 2 Precinct 02',
       'Leawood Ward 2 Precinct 03', 'Leawood Ward 2 Precinct 04',
       'Leawood Ward 2 Precinct 05', 'Leawood Ward 2 Precinct 06',
       'Leawood Ward 2 Precinct 07', 'Leawood Ward 3 Precinct 01',
       'Leawood Ward 3 Precinct 02', 'Leawood Ward 3 Precinct 03',
       'Leawood Ward 3 Precinct 04', 'Leawood Ward 3 Precinct 05',
       'Leawood Ward 3 Precinct 06', 'Leawood Ward 3 Precinct 07',
       'Leawood Ward 4 Precinct 01', 'Leawood Ward 4 Precinct 02',
       'Leawood Ward 4 Precinct 03', 'Leawood Ward 4 Precinct 04',
       'Leawood Ward 4 Precinct 05', 'Leawood Ward 4 Precinct 06',
       'Leawood Ward 4 Precinct 07', 'Lenexa Ward 1 Precinct 01',
       'Lenexa Ward 1 Precinct 02', 'Lenexa Ward 1 Precinct 03',
       'Lenexa Ward 1 Precinct 04', 'Lenexa Ward 1 Precinct 05',
       'Lenexa Ward 1 Precinct 06', 'Lenexa Ward 1 Precinct 07',
       'Lenexa Ward 1 Precinct 08', 'Lenexa Ward 1 Precinct 09',
       'Lenexa Ward 1 Precinct 10', 'Lenexa Ward 1 Precinct 11',
       'Lenexa Ward 2 Precinct 01', 'Lenexa Ward 2 Precinct 02',
       'Lenexa Ward 2 Precinct 03', 'Lenexa Ward 2 Precinct 04',
       'Lenexa Ward 2 Precinct 05', 'Lenexa Ward 2 Precinct 06',
       'Lenexa Ward 2 Precinct 07', 'Lenexa Ward 2 Precinct 08',
       'Lenexa Ward 2 Precinct 09', 'Lenexa Ward 2 Precinct 10',
       'Lenexa Ward 2 Precinct 11', 'Lenexa Ward 3 Precinct 01',
       'Lenexa Ward 3 Precinct 02', 'Lenexa Ward 3 Precinct 03',
       'Lenexa Ward 3 Precinct 04', 'Lenexa Ward 3 Precinct 05',
       'Lenexa Ward 3 Precinct 06', 'Lenexa Ward 3 Precinct 07',
       'Lenexa Ward 3 Precinct 08', 'Lenexa Ward 3 Precinct 09',
       'Lenexa Ward 3 Precinct 10', 'Lenexa Ward 4 Precinct 01',
       'Lenexa Ward 4 Precinct 02', 'Lenexa Ward 4 Precinct 03',
       'Lenexa Ward 4 Precinct 04', 'Lenexa Ward 4 Precinct 05',
       'Lenexa Ward 4 Precinct 06', 'Lenexa Ward 4 Precinct 07',
       'Lenexa Ward 4 Precinct 08', 'Lenexa Ward 4 Precinct 09',
       'Lenexa Ward 4 Precinct 10', 'Lenexa Ward 4 Precinct 11',
       'Lenexa Ward 4 Precinct 12', 'Lenexa Ward 4 Precinct 13',
       'Lenexa Ward 4 Precinct 14', 'Lenexa Ward 4 Precinct 15',
       'Merriam Ward 1 Precinct 01', 'Merriam Ward 1 Precinct 02',
       'Merriam Ward 1 Precinct 03', 'Merriam Ward 1 Precinct 04',
       'Merriam Ward 1 Precinct 05', 'Merriam Ward 2 Precinct 01',
       'Merriam Ward 2 Precinct 02', 'Merriam Ward 2 Precinct 03',
       'Merriam Ward 2 Precinct 04', 'Merriam Ward 3 Precinct 01',
       'Merriam Ward 3 Precinct 02', 'Merriam Ward 4 Precinct 01',
       'Merriam Ward 4 Precinct 02', 'Merriam Ward 4 Precinct 03',
       'Mission Ward 1 Precinct 01', 'Mission Ward 1 Precinct 02',
       'Mission Ward 2 Precinct 01', 'Mission Ward 2 Precinct 02',
       'Mission Ward 3 Precinct 01', 'Mission Ward 3 Precinct 02',
       'Mission Ward 4 Precinct 01', 'Mission Ward 4 Precinct 02',
       'Mission Ward 4 Precinct 03', 'Mission Ward 4 Precinct 04',
       'Mission Hills Precinct 01', 'Mission Hills Precinct 02',
       'Mission Hills Precinct 03', 'Mission Hills Precinct 04',
       'Mission Woods 0-01', 'Overland Park Ward 1 Precinct 01',
       'Overland Park Ward 1 Precinct 02',
       'Overland Park Ward 1 Precinct 03',
       'Overland Park Ward 1 Precinct 04',
       'Overland Park Ward 1 Precinct 05',
       'Overland Park Ward 1 Precinct 06',
       'Overland Park Ward 1 Precinct 07',
       'Overland Park Ward 1 Precinct 08',
       'Overland Park Ward 1 Precinct 09',
       'Overland Park Ward 1 Precinct 10',
       'Overland Park Ward 1 Precinct 11',
       'Overland Park Ward 1 Precinct 12',
       'Overland Park Ward 1 Precinct 13',
       'Overland Park Ward 1 Precinct 14',
       'Overland Park Ward 1 Precinct 15',
       'Overland Park Ward 1 Precinct 16',
       'Overland Park Ward 1 Precinct 17',
       'Overland Park Ward 1 Precinct 18',
       'Overland Park Ward 1 Precinct 19',
       'Overland Park Ward 1 Precinct 20',
       'Overland Park Ward 1 Precinct 21',
       'Overland Park Ward 1 Precinct 22',
       'Overland Park Ward 1 Precinct 23',
       'Overland Park Ward 1 Precinct 24',
       'Overland Park Ward 1 Precinct 25',
       'Overland Park Ward 1 Precinct 26',
       'Overland Park Ward 1 Precinct 27',
       'Overland Park Ward 2 Precinct 01',
       'Overland Park Ward 2 Precinct 02',
       'Overland Park Ward 2 Precinct 03',
       'Overland Park Ward 2 Precinct 04',
       'Overland Park Ward 2 Precinct 05',
       'Overland Park Ward 2 Precinct 06',
       'Overland Park Ward 2 Precinct 07',
       'Overland Park Ward 2 Precinct 08',
       'Overland Park Ward 2 Precinct 09',
       'Overland Park Ward 2 Precinct 10',
       'Overland Park Ward 2 Precinct 11',
       'Overland Park Ward 2 Precinct 12',
       'Overland Park Ward 2 Precinct 13',
       'Overland Park Ward 2 Precinct 14',
       'Overland Park Ward 2 Precinct 15',
       'Overland Park Ward 2 Precinct 16',
       'Overland Park Ward 2 Precinct 17',
       'Overland Park Ward 2 Precinct 18',
       'Overland Park Ward 2 Precinct 19',
       'Overland Park Ward 2 Precinct 20',
       'Overland Park Ward 2 Precinct 21',
       'Overland Park Ward 2 Precinct 22',
       'Overland Park Ward 2 Precinct 23',
       'Overland Park Ward 2 Precinct 24',
       'Overland Park Ward 2 Precinct 25',
       'Overland Park Ward 2 Precinct 26',
       'Overland Park Ward 2 Precinct 27',
       'Overland Park Ward 2 Precinct 28',
       'Overland Park Ward 2 Precinct 29',
       'Overland Park Ward 2 Precinct 30',
       'Overland Park Ward 2 Precinct 31',
       'Overland Park Ward 3 Precinct 01',
       'Overland Park Ward 3 Precinct 02',
       'Overland Park Ward 3 Precinct 03',
       'Overland Park Ward 3 Precinct 04',
       'Overland Park Ward 3 Precinct 05',
       'Overland Park Ward 3 Precinct 06',
       'Overland Park Ward 3 Precinct 07',
       'Overland Park Ward 3 Precinct 08',
       'Overland Park Ward 3 Precinct 09',
       'Overland Park Ward 3 Precinct 10',
       'Overland Park Ward 3 Precinct 11',
       'Overland Park Ward 3 Precinct 12',
       'Overland Park Ward 3 Precinct 13',
       'Overland Park Ward 3 Precinct 14',
       'Overland Park Ward 3 Precinct 15',
       'Overland Park Ward 3 Precinct 16',
       'Overland Park Ward 3 Precinct 17',
       'Overland Park Ward 3 Precinct 18',
       'Overland Park Ward 3 Precinct 19',
       'Overland Park Ward 3 Precinct 20',
       'Overland Park Ward 3 Precinct 21',
       'Overland Park Ward 3 Precinct 22',
       'Overland Park Ward 3 Precinct 23',
       'Overland Park Ward 3 Precinct 24',
       'Overland Park Ward 3 Precinct 25',
       'Overland Park Ward 3 Precinct 26',
       'Overland Park Ward 4 Precinct 01',
       'Overland Park Ward 4 Precinct 02',
       'Overland Park Ward 4 Precinct 03',
       'Overland Park Ward 4 Precinct 04',
       'Overland Park Ward 4 Precinct 05',
       'Overland Park Ward 4 Precinct 06',
       'Overland Park Ward 4 Precinct 07',
       'Overland Park Ward 4 Precinct 08',
       'Overland Park Ward 4 Precinct 09',
       'Overland Park Ward 4 Precinct 10',
       'Overland Park Ward 4 Precinct 11',
       'Overland Park Ward 4 Precinct 12',
       'Overland Park Ward 4 Precinct 13',
       'Overland Park Ward 4 Precinct 14',
       'Overland Park Ward 4 Precinct 15',
       'Overland Park Ward 4 Precinct 16',
       'Overland Park Ward 4 Precinct 17',
       'Overland Park Ward 4 Precinct 18',
       'Overland Park Ward 4 Precinct 19',
       'Overland Park Ward 4 Precinct 20',
       'Overland Park Ward 4 Precinct 21',
       'Overland Park Ward 4 Precinct 22',
       'Overland Park Ward 5 Precinct 01',
       'Overland Park Ward 5 Precinct 02',
       'Overland Park Ward 5 Precinct 03',
       'Overland Park Ward 5 Precinct 04',
       'Overland Park Ward 5 Precinct 05',
       'Overland Park Ward 5 Precinct 06',
       'Overland Park Ward 5 Precinct 07',
       'Overland Park Ward 5 Precinct 08',
       'Overland Park Ward 5 Precinct 09',
       'Overland Park Ward 5 Precinct 10',
       'Overland Park Ward 5 Precinct 11',
       'Overland Park Ward 5 Precinct 12',
       'Overland Park Ward 5 Precinct 13',
       'Overland Park Ward 5 Precinct 14',
       'Overland Park Ward 5 Precinct 15',
       'Overland Park Ward 5 Precinct 16',
       'Overland Park Ward 5 Precinct 17',
       'Overland Park Ward 5 Precinct 18',
       'Overland Park Ward 5 Precinct 19',
       'Overland Park Ward 5 Precinct 20',
       'Overland Park Ward 6 Precinct 01',
       'Overland Park Ward 6 Precinct 02',
       'Overland Park Ward 6 Precinct 03',
       'Overland Park Ward 6 Precinct 04',
       'Overland Park Ward 6 Precinct 05',
       'Overland Park Ward 6 Precinct 06',
       'Overland Park Ward 6 Precinct 07',
       'Overland Park Ward 6 Precinct 08',
       'Overland Park Ward 6 Precinct 09',
       'Overland Park Ward 6 Precinct 10',
       'Overland Park Ward 6 Precinct 11',
       'Overland Park Ward 6 Precinct 12',
       'Overland Park Ward 6 Precinct 13',
       'Overland Park Ward 6 Precinct 14',
       'Overland Park Ward 6 Precinct 15',
       'Overland Park Ward 6 Precinct 16',
       'Overland Park Ward 6 Precinct 17',
       'Overland Park Ward 6 Precinct 18',
       'Overland Park Ward 6 Precinct 19',
       'Overland Park Ward 6 Precinct 20',
       'Overland Park Ward 6 Precinct 21',
       'Overland Park Ward 6 Precinct 22',
       'Overland Park Ward 6 Precinct 23',
       'Overland Park Ward 6 Precinct 24',
       'Overland Park Ward 6 Precinct 25',
       'Prairie Village Ward 1 Precinct 01',
       'Prairie Village Ward 1 Precinct 02',
       'Prairie Village Ward 1 Precinct 03',
       'Prairie Village Ward 1 Precinct 04',
       'Prairie Village Ward 2 Precinct 01',
       'Prairie Village Ward 2 Precinct 02',
       'Prairie Village Ward 2 Precinct 03',
       'Prairie Village Ward 3 Precinct 01',
       'Prairie Village Ward 3 Precinct 02',
       'Prairie Village Ward 3 Precinct 03',
       'Prairie Village Ward 4 Precinct 01',
       'Prairie Village Ward 4 Precinct 02',
       'Prairie Village Ward 4 Precinct 03',
       'Prairie Village Ward 5 Precinct 01',
       'Prairie Village Ward 5 Precinct 02',
       'Prairie Village Ward 5 Precinct 03',
       'Prairie Village Ward 6 Precinct 01',
       'Prairie Village Ward 6 Precinct 02',
       'Prairie Village Ward 6 Precinct 03',
       'Roeland Park Ward 1 Precinct 01',
       'Roeland Park Ward 1 Precinct 02',
       'Roeland Park Ward 2 Precinct 01',
       'Roeland Park Ward 2 Precinct 02',
       'Roeland Park Ward 3 Precinct 01',
       'Roeland Park Ward 3 Precinct 02',
       'Roeland Park Ward 3 Precinct 03',
       'Roeland Park Ward 4 Precinct 01',
       'Roeland Park Ward 4 Precinct 02',
       'Roeland Park Ward 4 Precinct 03', 'Shawnee Ward 1 Precinct 01',
       'Shawnee Ward 1 Precinct 02', 'Shawnee Ward 1 Precinct 03',
       'Shawnee Ward 1 Precinct 04', 'Shawnee Ward 1 Precinct 05',
       'Shawnee Ward 1 Precinct 06', 'Shawnee Ward 1 Precinct 07',
       'Shawnee Ward 1 Precinct 08', 'Shawnee Ward 1 Precinct 09',
       'Shawnee Ward 1 Precinct 10', 'Shawnee Ward 1 Precinct 11',
       'Shawnee Ward 1 Precinct 12', 'Shawnee Ward 1 Precinct 13',
       'Shawnee Ward 1 Precinct 14', 'Shawnee Ward 2 Precinct 01',
       'Shawnee Ward 2 Precinct 02', 'Shawnee Ward 2 Precinct 03',
       'Shawnee Ward 2 Precinct 04', 'Shawnee Ward 2 Precinct 05',
       'Shawnee Ward 2 Precinct 06', 'Shawnee Ward 2 Precinct 07',
       'Shawnee Ward 2 Precinct 08', 'Shawnee Ward 2 Precinct 09',
       'Shawnee Ward 2 Precinct 10', 'Shawnee Ward 2 Precinct 11',
       'Shawnee Ward 2 Precinct 12', 'Shawnee Ward 2 Precinct 13',
       'Shawnee Ward 3 Precinct 01', 'Shawnee Ward 3 Precinct 02',
       'Shawnee Ward 3 Precinct 03', 'Shawnee Ward 3 Precinct 04',
       'Shawnee Ward 3 Precinct 05', 'Shawnee Ward 3 Precinct 06',
       'Shawnee Ward 3 Precinct 07', 'Shawnee Ward 3 Precinct 08',
       'Shawnee Ward 3 Precinct 09', 'Shawnee Ward 3 Precinct 10',
       'Shawnee Ward 3 Precinct 11', 'Shawnee Ward 3 Precinct 12',
       'Shawnee Ward 4 Precinct 01', 'Shawnee Ward 4 Precinct 02',
       'Shawnee Ward 4 Precinct 03', 'Shawnee Ward 4 Precinct 04',
       'Shawnee Ward 4 Precinct 05', 'Shawnee Ward 4 Precinct 06',
       'Shawnee Ward 4 Precinct 07', 'Shawnee Ward 4 Precinct 08',
       'Shawnee Ward 4 Precinct 09', 'Shawnee Ward 4 Precinct 10',
       'Shawnee Ward 4 Precinct 11', 'Shawnee Ward 4 Precinct 12',
       'Shawnee Ward 4 Precinct 13', 'Shawnee Ward 4 Precinct 14',
       'Shawnee Ward 4 Precinct 15', 'Westwood Precinct 01',
       'Westwood Precinct 02', 'Westwood Hills Precinct 01',
       'Lake Quivira Precinct 01', 'Bonner Springs Ward 4 Precinct 01']

sorted_list1 = sorted(list1)
sorted_list2 = sorted(list2)

df_comparison = pd.DataFrame({
    'List 1': sorted_list1,
    'List 2': sorted_list2
})
pd.set_option('display.max_rows', None)

df_comparison.head()

,List 1,List 2
0,Aubry Twp 0-01,Aubry Township Precinct 01
1,Aubry Twp 0-02,Aubry Township Precinct 02
2,Aubry Twp 0-03,Aubry Township Precinct 03
3,Aubry Twp 0-04,Aubry Township Precinct 04
4,Aubry Twp 0-05,Aubry Township Precinct 05


Create a replacement dictionary to replace precinct names in election reults with those in PBER so that they will match perfectly when merge

In [65]:
replacement_dict = dict(zip(sorted_list1, sorted_list2))

modified_precinct_dict = {
    (key.replace('Olathe Ward', 'Olathe') if key.startswith('Olathe Ward') else key): value
    for key, value in replacement_dict.items()
}
modified_precinct_dict['Westwood 0-01'] = 'Westwood Precinct 01'
modified_precinct_dict['Westwood 0-02'] = 'Westwood Precinct 02'
modified_precinct_dict['Westwood Hills 0-01'] = 'Westwood Hills Precinct 01'

modified_precinct_dict = {value: key for key, value in modified_precinct_dict.items()}

modified_precinct_dict

{'Aubry Township Precinct 01': 'Aubry Twp 0-01',
 'Aubry Township Precinct 02': 'Aubry Twp 0-02',
 'Aubry Township Precinct 03': 'Aubry Twp 0-03',
 'Aubry Township Precinct 04': 'Aubry Twp 0-04',
 'Aubry Township Precinct 05': 'Aubry Twp 0-05',
 'Aubry Township Precinct 06': 'Aubry Twp 0-06',
 'Aubry Township Precinct 07': 'Aubry Twp 0-07',
 'Aubry Township Precinct 08': 'Aubry Twp 0-08',
 'Bonner Springs Ward 4 Precinct 01': 'Bonner Springs 4-01',
 'De Soto Precinct 01': 'De Soto 0-01',
 'De Soto Precinct 02': 'De Soto 0-02',
 'De Soto Precinct 03': 'De Soto 0-03',
 'De Soto Precinct 04': 'De Soto 0-04',
 'De Soto Precinct 05': 'De Soto 0-05',
 'De Soto Precinct 06': 'De Soto 0-06',
 'De Soto Precinct 07': 'De Soto 0-07',
 'De Soto Precinct 08': 'De Soto 0-08',
 'De Soto Precinct 09': 'De Soto 0-09',
 'De Soto Precinct 10': 'De Soto 0-10',
 'De Soto Precinct 11': 'De Soto 0-11',
 'De Soto Precinct 12': 'De Soto 0-12',
 'Edgerton Precinct 01': 'Edgerton 0-01',
 'Edgerton Precinct 02': 

Replace and change format

In [66]:
JOHNSON_COUNTY.head()


,PRECINCT CODE,PRECINCT NAME,VTD,Unnamed: 3,Kansas Representative 5th District,Kansas Representative 5th District.1,Kansas Representative 8th District,Kansas Representative 8th District.1,Kansas Representative 8th District.2,Kansas Representative 14th District,...,Kansas Representative 78th District.1,Kansas Representative 78th District.2,Kansas Representative 108th District,Kansas Representative 108th District.1,Kansas Representative 117th District,Kansas Representative 117th District.1,Kansas Representative 117th District.2,Kansas Representative 121st District,Kansas Representative 121st District.1,Kansas Representative 121st District.2
0,NaN,NaN,NaN,NaN,Carrie Barth,Write-in,Chris Croft,Pam Shernuk,Write-in,Charlotte Esau,...,W. Michael Shimeall,Write-in,Brandon Woodard,Write-in,Courtney Tripp,Adam Turk,Write-in,Mel Pinick,John Resman,Write-in
1,1,Aubry Township Precinct 01,10,1001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Aubry Township Precinct 02,20,1002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Aubry Township Precinct 03,220010,1003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Aubry Township Precinct 04,40,1004,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
JOHNSON_COUNTY['PRECINCT NAME'] = JOHNSON_COUNTY['PRECINCT NAME'].replace(modified_precinct_dict)
len(JOHNSON_COUNTY)


612

In [68]:
candidate_names = JOHNSON_COUNTY.iloc[0, 4:]

johnson_county_cleaned = JOHNSON_COUNTY.iloc[1:].reset_index(drop=True)

results = []
for idx, row in johnson_county_cleaned.iterrows():
    precinct_code = row['PRECINCT CODE']
    precinct_name = row['PRECINCT NAME']
    vtd = row['VTD']
    
    for col in candidate_names.index:
        if row[col] > 0:
            candidate = candidate_names[col] 
            votes = row[col]
            race = col.split('.')[0] 
            
            results.append({
                'County': 'Johnson',
                'Precinct': precinct_name,
                'Race': race,
                'Candidate': candidate,
                'Votes': votes,
                'VTD': vtd
            })

results_df_JOHNSON_COUNTY = pd.DataFrame(results)

results_df_JOHNSON_COUNTY.head(5)
len(results_df_JOHNSON_COUNTY)

1229

In [69]:
results_df_JOHNSON_COUNTY['Precinct'].nunique()

542

Election results is missing candidate's party information, so I did some scripting to get it.

In [70]:
# Creating a dictionary with candidates and their party affiliation
candidates_party = {
    "Mark R. Holland": "Democratic",
    "Jerry Moran": "Republican",
    "David C Graham": "Libertarian",
    "James 'Jimmy' Beard": "Democratic",
    "Tracey Mann": "Republican",
    "Patrick Schmidt": "Democratic",
    "Jake LaTurner": "Republican",
    "Sharice Davids": "Democratic",
    "Amanda L. Adkins": "Republican",
    "Steven A Hohe": "Libertarian",
    "Bob Hernandez": "Democratic",
    "Ron Estes": "Republican",
    "Laura Kelly / David Toland": "Democratic",
    "Derek Schmidt / Katie Sawyer": "Republican",
    "Seth Cordell / Evan Laudick-Gains": "Libertarian",
    "Dennis Pyle / Kathleen E. Garrison": "Independent",
    "Jeanna Repass": "Democratic",
    "Scott J Schwab": "Republican",
    "Cullene Lang": "Libertarian",
    "Chris Mann": "Democratic",
    "Kris Kobach": "Republican",
    "Lynn W Rogers": "Democratic",
    "Steven Johnson": "Republican",
    "Steve Roberts": "Libertarian",
    "Kiel Corkran": "Democratic",
    "Vicki Schmidt": "Republican",
    "Jose Lara": "Democratic",
    "Ronald W. Ryckman Sr": "Republican",
    "Paul D. Rogers": "Democratic",
    "Michael Houser": "Republican",
    "Kenneth (Ken) Collins": "Republican",
    "Chuck Smith": "Republican",
    "Trevor Jacobs": "Republican",
    "Carrie Barth": "Republican",
    "Nina Fricke": "Democratic",
    "Samantha M. Poetter Parshall": "Republican",
    "Dan Goddard": "Republican",
    "Pam Shernuk": "Democratic",
    "Chris Croft": "Republican",
    "Alana Cloutier": "Democratic",
    "Fred Gardner": "Republican",
    "Christina Haswood": "Democratic",
    "Gregory L. Wilkinson": "Democratic",
    "Ron Bryce": "Republican",
    "Doug Blex": "Republican",
    "Joe Newland": "Republican",
    "Dennis Miller": "Democratic",
    "Charlotte Esau": "Republican",
    "Allison Hougland": "Democratic",
    "Matt Bingesser": "Republican",
    "Linda Featherston": "Democratic",
    "Ed Roitz": "Republican",
    "Jo Ella Hoye": "Democratic",
    "Emily Carpenter": "Republican",
    "Michael Kerner": "Libertarian",
    "Cindy Neighbor": "Democratic",
    "Cathy Gordon": "Republican",
    "Stephanie Sawyer Clayton": "Democratic",
    "Nicholas Reddell": "Republican",
    "Mari-Lynn Poskin": "Democratic",
    "Carrie Rahfaldt": "Republican",
    "Jerry Stogsdill": "Democratic",
    "Lindsay Vaughn": "Democratic",
    "Robert 'Chris' Colburn": "Republican",
    "Susan Ruiz": "Democratic",
    "Jarrod Ousley": "Democratic",
    "Rui Xu": "Democratic",
    "Cheron Tiffany": "Democratic",
    "Adam T. Thomas": "Republican",
    "Christi Pribula": "Democratic",
    "Sean Edward Tarwater Sr.": "Republican",
    "Ace Allen": "Democratic",
    "Carl Turner": "Republican",
    "Heather Meyer": "Democratic",
    "David Soffer": "Republican",
    "Courtney Eiterich": "Democratic",
    "Laura Williams": "Republican",
    "Louis E. Ruiz": "Democratic",
    "Dennis Grindel": "Republican",
    "Pam Curtis": "Democratic",
    "Bill Hutton": "Democratic",
    "Mike Thompson": "Republican",
    "Stephanie Barton": "Libertarian",
    "Valdenia C. Winn": "Democratic",
    "Pepe Cabrera": "Republican",
    "Marvin S. Robinson II": "Democratic",
    "Sam Stillwell": "Republican",
    "Lynn Melton": "Democratic",
    "Kevin Braun": "Republican",
    "Melissa Oropeza": "Democratic",
    "Diana Whittington": "Republican",
    "Timothy H. Johnson": "Republican",
    "Vanessa Vaughn West": "Democratic",
    "Owen Donohoe": "Republican",
    "Martha Allen": "Democratic",
    "David Buehler": "Republican",
    "Harry Schwarz": "Democratic",
    "Pat Proctor": "Republican",
    "Lance W. Neelly": "Republican",
    "Keith Davenport": "Democratic",
    "Bill Sutton": "Republican",
    "Barbara W. Ballard": "Democratic",
    "Mike Amyx": "Democratic",
    "Dennis 'Boog' Highberger": "Democratic",
    "Ronald Ellis": "Republican",
    "Dan Osman": "Democratic",
    "Terry Frederick": "Republican",
    "Brad Boyd": "Democratic",
    "Kristin A. Clark": "Republican",
    "Fred C. Patton": "Republican",
    "Kenny Titus": "Republican",
    "Derik Flerlage": "Democratic",
    "Jesse Borjon": "Republican",
    "Kirk R. Haskins": "Democratic",
    "Bruce Williamson": "Republican",
    "Lyndon Johnson": "Democratic",
    "Ken Corbet": "Republican",
    "Tobias Schlingensiepen": "Democratic",
    "Todd C. Staerkel": "Republican",
    "Virgil Weigel": "Democratic",
    "John Alcala": "Democratic",
    "Vic Miller": "Democratic",
    "Darrell McCune": "Democratic",
    "Rebecca Schmoe": "Republican",
    "Mic McGuire": "Democratic",
    "Mark Schreiber": "Republican",
    "Francis Awerkamp": "Republican",
    "Randy Garber": "Republican",
    "John R. Eplee": "Republican",
    "Patricia Smetana": "Democratic",
    "Lewis (Bill) Bloom": "Republican",
    "Jeff Underhill": "Republican",
    "Sydney Carlin": "Democratic",
    "Kim Zito": "Democratic",
    "Mike Dodson": "Republican",
    "Michael Seymour II": "Democratic",
    "Nathan (Nate) Butler": "Republican",
    "Sarah Crews": "Democratic",
    "Clarke Sanders": "Republican",
    "Scott Hill": "Republican",
    "Steven K. Howe": "Republican",
    "Avery Anderson": "Republican",
    "Les Mason": "Republican",
    "Stephen Owens": "Republican",
    "Henry Hein": "Libertarian",
    "Will Carpenter": "Republican",
    "Chuck Torres": "Democratic",
    "Eric L. Smith": "Republican",
    "Kristey S. Williams": "Republican",
    "W. Michael Shimeall": "Democratic",
    "Robyn R. Essex": "Republican",
    "Kris Trimmer": "Democratic",
    "Webster T. Roth": "Republican",
    "Bill Rhiley": "Republican",
    "Blake Carpenter": "Republican",
    "Misti Hobbs": "Democratic",
    "Leah Howell": "Republican",
    "Henry Helgerson": "Democratic",
    "Ford Carr": "Democratic",
    "Jalon Britton": "Democratic",
    "Patrick A. Penn": "Republican",
    "Silas Miller": "Democratic",
    "Rick Lindsey": "Republican",
    "Chris Strong": "Democratic",
    "Susan Oliver Estes": "Republican",
    "Chuck Schmidt": "Democratic",
    "Sandy Pickert": "Republican",
    "KC Ohaebosim": "Democratic",
    "Carl Maughan": "Republican",
    "Brooke Chong": "Democratic",
    "Emil M. Bergquist": "Republican",
}

df_candidates_party = pd.DataFrame(list(candidates_party.items()), columns=['Candidate', 'Party'])
merged_df = pd.merge(results_df_JOHNSON_COUNTY, df_candidates_party[['Candidate', 'Party']], how='left', on='Candidate', indicator=True)
merged_df['_merge'].value_counts()

_merge
both          845
left_only     384
right_only      0
Name: count, dtype: int64

In [71]:
merged_df.head()

,County,Precinct,Race,Candidate,Votes,VTD,Party,_merge
0,Johnson,Aubry Twp 0-01,Kansas Representative 27th District,Christi Pribula,51,10,Democratic,both
1,Johnson,Aubry Twp 0-01,Kansas Representative 27th District,"Sean Edward Tarwater, Sr.",71,10,NaN,left_only
2,Johnson,Aubry Twp 0-02,Kansas Representative 27th District,Christi Pribula,258,20,Democratic,both
3,Johnson,Aubry Twp 0-02,Kansas Representative 27th District,"Sean Edward Tarwater, Sr.",496,20,NaN,left_only
4,Johnson,Aubry Twp 0-03,Kansas Representative 27th District,Christi Pribula,120,220010,Democratic,both


In [72]:
unique_candidates = merged_df[['Candidate', 'Party']].drop_duplicates()
unique_candidates.head()

,Candidate,Party
0,Christi Pribula,Democratic
1,"Sean Edward Tarwater, Sr.",NaN
8,Adam T. Thomas,Republican
9,Cheron Tiffany,Democratic
14,Keith Davenport,Democratic


In [73]:
replacementDict = {
    'Sean Edward Tarwater, Sr.': 'Sean Tarwater',
    'Robert "Chris" Colburn': 'Robert Colburn'
}

partyDict = {
    'Sean Edward Tarwater, Sr.': 'Sean Tarwater',
    'Robert "Chris" Colburn': 'Robert Colburn'
}

merged_df['Candidate'] = merged_df['Candidate'].replace(replacementDict)

merged_df.loc[merged_df['Candidate'] == 'Sean Tarwater', 'Party'] = 'Republican'
merged_df.loc[merged_df['Candidate'] == 'Write-in', 'Party'] = 'Other'

merged_df.loc[merged_df['Candidate'] == 'Mel Pinick', 'Party'] = 'Democratic'

merged_df.loc[merged_df['Candidate'] == 'John Resman', 'Party'] = 'Republican'

merged_df.loc[merged_df['Candidate'] == 'Courtney Tripp', 'Party'] = 'Democratic'
merged_df.loc[merged_df['Candidate'] == 'Courtney Tripp', 'Party'] = 'Democratic'
merged_df.loc[merged_df['Candidate'] == 'Adam Turk', 'Party'] = 'Republican'
merged_df.loc[merged_df['Candidate'] == 'Brandon Woodard', 'Party'] = 'Democratic'
merged_df.loc[merged_df['Candidate'] == 'Robert Colburn', 'Party'] = 'Republican'


merged_df = merged_df.drop('_merge', axis=1)

merged_df.head()


,County,Precinct,Race,Candidate,Votes,VTD,Party
0,Johnson,Aubry Twp 0-01,Kansas Representative 27th District,Christi Pribula,51,10,Democratic
1,Johnson,Aubry Twp 0-01,Kansas Representative 27th District,Sean Tarwater,71,10,Republican
2,Johnson,Aubry Twp 0-02,Kansas Representative 27th District,Christi Pribula,258,20,Democratic
3,Johnson,Aubry Twp 0-02,Kansas Representative 27th District,Sean Tarwater,496,20,Republican
4,Johnson,Aubry Twp 0-03,Kansas Representative 27th District,Christi Pribula,120,220010,Democratic


In [74]:
results_df_JOHNSON_COUNTY = merged_df[['County', 'Precinct', 'Race', 'Candidate', 'Party', 'Votes', 'VTD']]
results_df_JOHNSON_COUNTY.tail()

,County,Precinct,Race,Candidate,Party,Votes,VTD
1224,Johnson,COUNTY TOTALS,Kansas Representative 117th District,Adam Turk,Republican,4671,NaN
1225,Johnson,COUNTY TOTALS,Kansas Representative 117th District,Write-in,Other,4,NaN
1226,Johnson,COUNTY TOTALS,Kansas Representative 121st District,Mel Pinick,Democratic,4133,NaN
1227,Johnson,COUNTY TOTALS,Kansas Representative 121st District,John Resman,Republican,4972,NaN
1228,Johnson,COUNTY TOTALS,Kansas Representative 121st District,Write-in,Other,7,NaN


In [75]:
county_totals_df_Johnson = results_df_JOHNSON_COUNTY[results_df_JOHNSON_COUNTY['Precinct'] == 'COUNTY TOTALS']

results_df_JOHNSON_COUNTY = results_df_JOHNSON_COUNTY[results_df_JOHNSON_COUNTY['Precinct'] != 'COUNTY TOTALS']
results_df_JOHNSON_COUNTY.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Johnson,Aubry Twp 0-01,Kansas Representative 27th District,Christi Pribula,Democratic,51,10
1,Johnson,Aubry Twp 0-01,Kansas Representative 27th District,Sean Tarwater,Republican,71,10
2,Johnson,Aubry Twp 0-02,Kansas Representative 27th District,Christi Pribula,Democratic,258,20
3,Johnson,Aubry Twp 0-02,Kansas Representative 27th District,Sean Tarwater,Republican,496,20
4,Johnson,Aubry Twp 0-03,Kansas Representative 27th District,Christi Pribula,Democratic,120,220010


In [76]:
len(results_df_JOHNSON_COUNTY)

1153

In [77]:
results_df_JOHNSON_COUNTY['VTD'].nunique()

541

### Process SHAWNEE COUNTY

In [78]:
Shawnee_PBER = gdf_kansan_2022[gdf_kansan_2022['COUNTYNA'] == 'Shawnee']
Shawnee_PBER.head()
Shawnee_PBER['NAMELSAD'].unique()

array(['East Peck Voting District', 'East Rossville Voting District',
       'Grove Voting District', 'Topeka Ward 3 Precinct 08',
       'Topeka Ward 6 Precinct 08', 'Topeka Ward 8 Precinct 05',
       'Kiowa Voting District', 'North Tecumseh Voting District',
       'Topeka Ward 1 Precinct 01', 'Topeka Ward 2 Precinct 10',
       'Topeka Ward 5 Precinct 12', 'North Potter Voting District',
       'Northeast Monmouth Voting District',
       'Northwest Monmouth Voting District',
       'South Monmouth Voting District', 'Pawnee Voting District',
       'Peck/Croco Voting District', 'Ponca Voting District',
       'Rochester Voting District', 'Sherwood Voting District',
       'Sioux Voting District', 'Topeka Ward 1 Precinct 02',
       'Topeka Ward 2 Precinct 04', 'Topeka Ward 5 Precinct 91',
       'Topeka Ward 1 Precinct 03', 'Topeka Ward 3 Precinct 01',
       'Topeka Ward 7 Precinct 01', 'Topeka Ward 1 Precinct 04',
       'Topeka Ward 2 Precinct 05', 'Topeka Ward 3 Precinct 02',
 

In [79]:
SHAWNEE_COUNTY['PRECINCT NAME'].unique()
# SHAWNEE_COUNTY.head(5)

array([nan, '1 East Rossville', '2 West Rossville', '9 East Silver Lake',
       '10 West Silver Lake', '12 Grove', '14 Kiro', '15 Messhoss Creek',
       '16 SE Kiro', '17 Croco', '18 Flathead', '22 Little Muddy Creek',
       '23 Coup', '24 Sac', '25 Sherman', '26 Iroquois', '27 Halfday',
       '28 West Wichita', '29 Indianola', '30 Apache', '31 East Wichita',
       '32 Fox', '33 Elmont', '34 Kilmer', '35 Rochester',
       '36 East Soldier', '37 Whitfield', '38 Sioux', '39 Pottawatomie',
       '40 Stinson Creek', '41 North Tecumseh', '42 South Tecumseh',
       '43 Pawnee', '44 Kaw', '45 Peck', '46 East Peck', '47 Kiowa',
       '48 Cheyenne', '49 Ponca', '50 North Potter', '51 Channel',
       '52 Deer Creek', '53 Pauline', '54 West Potter',
       '55 Central Pauline', '56 NW Monmouth', '57 NE Monmouth',
       '58 South Monmouth', '59 NC Monmouth', '61 Sixty-First St',
       '62 Forbes', '63 Montara', '64 Cullen', '65 Wakarusa',
       '66 Kingston', '72 City Of Auburn', '73 

In [80]:
list1_s = ['1 East Rossville', '2 West Rossville', '9 East Silver Lake',
       '10 West Silver Lake', '12 Grove', '14 Kiro', '15 Messhoss Creek',
       '16 SE Kiro', '17 Croco', '18 Flathead', '22 Little Muddy Creek',
       '23 Coup', '24 Sac', '25 Sherman', '26 Iroquois', '27 Halfday',
       '28 West Wichita', '29 Indianola', '30 Apache', '31 East Wichita',
       '32 Fox', '33 Elmont', '34 Kilmer', '35 Rochester',
       '36 East Soldier', '37 Whitfield', '38 Sioux', '39 Pottawatomie',
       '40 Stinson Creek', '41 North Tecumseh', '42 South Tecumseh',
       '43 Pawnee', '44 Kaw', '45 Peck', '46 East Peck', '47 Kiowa',
       '48 Cheyenne', '49 Ponca', '50 North Potter', '51 Channel',
       '52 Deer Creek', '53 Pauline', '54 West Potter',
       '55 Central Pauline', '56 NW Monmouth', '57 NE Monmouth',
       '58 South Monmouth', '59 NC Monmouth', '61 Sixty-First St',
       '62 Forbes', '63 Montara', '64 Cullen', '65 Wakarusa',
       '66 Kingston', '72 City Of Auburn', '73 Meadowlark', '74 Buffalo',
       '78 Hodges Rd', '79 North Gaillardia', '80 Gaillardia', '81 Dover',
       '82 Willard', '83 Ichabod Washburn', '84 Yucca', '85 Aldersgate',
       '86 Sunflower', '87 Washburn', '88 Lykins', '89 North Mission',
       '90 South Mission', '91 Central Mission', '92 Sherwood',
       '93 South Sherwood', '94 Indian Hills', '95 York', '96 Vaquero',
       '97 York Outer', '98 NE Mission', '99 Verbena',
       'Ward 1 Precinct 1', 'Ward 1 Precinct 2', 'Ward 1 Precinct 3',
       'Ward 1 Precinct 4', 'Ward 1 Precinct 5', 'Ward 1 Precinct 6',
       'Ward 1 Precinct 7', 'Ward 1 Precinct 8', 'Ward 1 Precinct 9',
       'Ward 1 Precinct 10', 'Ward 1 Precinct 11', 'Ward 1 Precinct 13',
       'Ward 1 Precinct 14', 'Ward 2 Precinct 1', 'Ward 2 Precinct 2',
       'Ward 2 Precinct 3', 'Ward 2 Precinct 4', 'Ward 2 Precinct 5',
       'Ward 2 Precinct 6', 'Ward 2 Precinct 7', 'Ward 2 Precinct 8',
       'Ward 2 Precinct 9', 'Ward 2 Precinct 10', 'Ward 2 Precinct 11',
       'Ward 2 Precinct 12', 'Ward 3 Precinct 1', 'Ward 3 Precinct 2',
       'Ward 3 Precinct 3', 'Ward 3 Precinct 4', 'Ward 3 Precinct 5',
       'Ward 3 Precinct 6', 'Ward 3 Precinct 7', 'Ward 3 Precinct 8',
       'Ward 3 Precinct 9', 'Ward 3 Precinct 10', 'Ward 4 Precinct 1',
       'Ward 4 Precinct 2', 'Ward 4 Precinct 3', 'Ward 4 Precinct 4',
       'Ward 4 Precinct 5', 'Ward 4 Precinct 6', 'Ward 4 Precinct 7',
       'Ward 4 Precinct 8', 'Ward 4 Precinct 9', 'Ward 4 Precinct 10',
       'Ward 4 Precinct 11', 'Ward 4 Precinct 12', 'Ward 4 Precinct 13',
       'Ward 4 Precinct 14', 'Ward 4 Precinct 15', 'Ward 5 Precinct 1',
       'Ward 5 Precinct 2', 'Ward 5 Precinct 3', 'Ward 5 Precinct 4',
       'Ward 5 Precinct 5', 'Ward 5 Precinct 6', 'Ward 5 Precinct 7',
       'Ward 5 Precinct 8', 'Ward 5 Precinct 9', 'Ward 5 Precinct 10',
       'Ward 5 Precinct 11', 'Ward 5 Precinct 12', 'Ward 5 Precinct 13',
       'Ward 5 Precinct 14', 'Ward 5 Precinct 15', 'Ward 5 Precinct 91',
       'Ward 6 Precinct 1', 'Ward 6 Precinct 2', 'Ward 6 Precinct 3',
       'Ward 6 Precinct 4', 'Ward 6 Precinct 5', 'Ward 6 Precinct 6',
       'Ward 6 Precinct 7', 'Ward 6 Precinct 8', 'Ward 6 Precinct 9',
       'Ward 6 Precinct 10', 'Ward 6 Precinct 11', 'Ward 6 Precinct 12',
       'Ward 6 Precinct 13', 'Ward 7 Precinct 1', 'Ward 7 Precinct 2',
       'Ward 7 Precinct 3', 'Ward 7 Precinct 4', 'Ward 7 Precinct 5',
       'Ward 7 Precinct 6', 'Ward 7 Precinct 7', 'Ward 7 Precinct 9',
       'Ward 7 Precinct 10', 'Ward 7 Precinct 11', 'Ward 8 Precinct 1',
       'Ward 8 Precinct 2', 'Ward 8 Precinct 3', 'Ward 8 Precinct 4',
       'Ward 8 Precinct 5', 'Ward 8 Precinct 6', 'Ward 8 Precinct 7',
       'Ward 8 Precinct 8', 'Ward 8 Precinct 9', 'Ward 8 Precinct 10',
       'Ward 8 Precinct 11', 'Ward 9 Precinct 1', 'Ward 9 Precinct 2',
       'Ward 9 Precinct 3', 'Ward 9 Precinct 4', 'Ward 9 Precinct 5',
       'Ward 9 Precinct 6', 'Ward 9 Precinct 8', 'Ward 9 Precinct 9',
       'Ward 9 Precinct 10', 'Ward 9 Precinct 11', 'Ward 9 Precinct 12',
       'Ward 10 Precinct 1', 'Ward 10 Precinct 2', 'Ward 10 Precinct 3',
       'Ward 10 Precinct 4', 'Ward 10 Precinct 5', 'Ward 10 Precinct 6',
       'Ward 10 Precinct 7', 'Ward 10 Precinct 8', 'Ward 10 Precinct 9',
       'Ward 10 Precinct 10', 'Ward 10 Precinct 11',
       'Ward 10 Precinct 12', 'Ward 10 Precinct 13',
       'Ward 10 Precinct 14', 'Ward 10 Precinct 15', 'Ward 11 Precinct 1',
       'Ward 11 Precinct 2', 'Ward 11 Precinct 3', 'Ward 11 Precinct 4',
       'Ward 11 Precinct 5', 'Ward 11 Precinct 6', 'Ward 11 Precinct 7',
       'Ward 11 Precinct 8', 'Ward 11 Precinct 9', 'Ward 11 Precinct 10',
       'Ward 11 Precinct 11', 'Ward 12 Precinct 1', 'Ward 12 Precinct 2',
       'Ward 12 Precinct 3', 'Ward 12 Precinct 4', 'Ward 12 Precinct 5',
       'Ward 12 Precinct 6', 'Ward 12 Precinct 7', 'Ward 12 Precinct 8',
       'Ward 12 Precinct 9', 'Ward 12 Precinct 11', 'Ward 12 Precinct 12',
       'Ward 12 Precinct 13', 'Ward 12 Precinct 14',
       'Ward 12 Precinct 15', 'Ward 12 Precinct 16',
       'Ward 12 Precinct 17', 'Ward 12 Precinct 19',
       'Ward 12 Precinct 20', 'Ward 12 Precinct 31',
       'Ward 12 Precinct 41', 'Ward 12 Precinct 81',
       'Ward 12 Precinct 91', 'Ward 12 Precinct 92',
       'Ward 12 Precinct 93', 'Ward 13 Precinct 1', 'Ward 13 Precinct 2',
       'Ward 13 Precinct 3', 'Ward 13 Precinct 4', 'Ward 13 Precinct 5',
       'Ward 13 Precinct 6', 'Ward 13 Precinct 7', 'Ward 13 Precinct 8',
       'Ward 13 Precinct 9', 'Ward 13 Precinct 10', 'Ward 13 Precinct 12',
       'Ward 13 Precinct 13', 'Ward 13 Precinct 21',
       'Ward 13 Precinct 22', 'Ward 13 Precinct 30', 'Ward 14 Precinct 1',
       'Ward 14 Precinct 2', 'Ward 14 Precinct 3', 'Ward 14 Precinct 4',
       'Ward 14 Precinct 5', 'Ward 15 Precinct 1', 'Ward 15 Precinct 2',
       'Ward 15 Precinct 3', 'Ward 15 Precinct 4']

list2_s = ['East Peck Voting District', 'East Rossville Voting District',
       'Grove Voting District', 'Topeka Ward 3 Precinct 08',
       'Topeka Ward 6 Precinct 08', 'Topeka Ward 8 Precinct 05',
       'Kiowa Voting District', 'North Tecumseh Voting District',
       'Topeka Ward 1 Precinct 01', 'Topeka Ward 2 Precinct 10',
       'Topeka Ward 5 Precinct 12', 'North Potter Voting District',
       'Northeast Monmouth Voting District',
       'Northwest Monmouth Voting District',
       'South Monmouth Voting District', 'Pawnee Voting District',
       'Peck/Croco Voting District', 'Ponca Voting District',
       'Rochester Voting District', 'Sherwood Voting District',
       'Sioux Voting District', 'Topeka Ward 1 Precinct 02',
       'Topeka Ward 2 Precinct 04', 'Topeka Ward 5 Precinct 91',
       'Topeka Ward 1 Precinct 03', 'Topeka Ward 3 Precinct 01',
       'Topeka Ward 7 Precinct 01', 'Topeka Ward 1 Precinct 04',
       'Topeka Ward 2 Precinct 05', 'Topeka Ward 3 Precinct 02',
       'Topeka Ward 2 Precinct 01', 'Topeka Ward 2 Precinct 11',
       'Topeka Ward 2 Precinct 02', 'Topeka Ward 2 Precinct 03',
       'Topeka Ward 3 Precinct 05', 'Topeka Ward 2 Precinct 06',
       'Topeka Ward 3 Precinct 07', 'Topeka Ward 2 Precinct 07',
       'Topeka Ward 2 Precinct 09', 'Topeka Ward 2 Precinct 08',
       'Topeka Ward 3 Precinct 03', 'Topeka Ward 3 Precinct 06',
       'Topeka Ward 4 Precinct 01', 'Topeka Ward 4 Precinct 02',
       'Topeka Ward 7 Precinct 02', 'Topeka Ward 4 Precinct 03',
       'Topeka Ward 4 Precinct 12', 'Topeka Ward 7 Precinct 07',
       'Topeka Ward 4 Precinct 06', 'Topeka Ward 4 Precinct 10',
       'Topeka Ward 6 Precinct 03', 'Topeka Ward 4 Precinct 07',
       'Topeka Ward 5 Precinct 04', 'Topeka Ward 5 Precinct 05',
       'Topeka Ward 4 Precinct 08', 'Topeka Ward 5 Precinct 09',
       'Topeka Ward 4 Precinct 09', 'Topeka Ward 6 Precinct 01',
       'Topeka Ward 5 Precinct 02', 'Topeka Ward 6 Precinct 05',
       'Topeka Ward 7 Precinct 09', 'Topeka Ward 5 Precinct 03',
       'Topeka Ward 6 Precinct 02', 'Topeka Ward 6 Precinct 07',
       'Topeka Ward 6 Precinct 10', 'Topeka Ward 5 Precinct 06',
       'Topeka Ward 6 Precinct 04', 'Topeka Ward 8 Precinct 01',
       'Topeka Ward 8 Precinct 02', 'Topeka Ward 5 Precinct 07',
       'Topeka Ward 5 Precinct 08', 'Topeka Ward 7 Precinct 03',
       'Topeka Ward 6 Precinct 06', 'Topeka Ward 7 Precinct 04',
       'Topeka Ward 7 Precinct 05', 'Topeka Ward 7 Precinct 06',
       'Topeka Ward 7 Precinct 11', 'Topeka Ward 8 Precinct 03',
       'Topeka Ward 8 Precinct 04', 'Topeka Ward 7 Precinct 10',
       'Topeka Ward 8 Precinct 06', 'Topeka Ward 8 Precinct 07',
       'Topeka Ward 8 Precinct 08', 'Topeka Ward 8 Precinct 09',
       'Topeka Ward 8 Precinct 11', 'Topeka Ward 9 Precinct 02',
       'Topeka Ward 9 Precinct 03', 'Topeka Ward 9 Precinct 04',
       'Topeka Ward 9 Precinct 05', 'Topeka Ward 9 Precinct 06',
       'Topeka Ward 9 Precinct 09', 'Topeka Ward 9 Precinct 10',
       'Topeka Ward 9 Precinct 11', 'Topeka Ward 10 Precinct 01',
       'Topeka Ward 10 Precinct 02', 'Topeka Ward 10 Precinct 03',
       'Topeka Ward 10 Precinct 06', 'Topeka Ward 10 Precinct 07',
       'Topeka Ward 10 Precinct 08', 'Topeka Ward 10 Precinct 09',
       'Topeka Ward 10 Precinct 10', 'Topeka Ward 11 Precinct 04',
       'Topeka Ward 11 Precinct 06', 'Topeka Ward 11 Precinct 08',
       'Topeka Ward 11 Precinct 10', 'Topeka Ward 12 Precinct 01',
       'Topeka Ward 12 Precinct 04', 'Topeka Ward 12 Precinct 05',
       'Topeka Ward 12 Precinct 06', 'Topeka Ward 12 Precinct 07',
       'Topeka Ward 12 Precinct 08', 'Topeka Ward 12 Precinct 09',
       'Topeka Ward 12 Precinct 11', 'Topeka Ward 12 Precinct 12',
       'Topeka Ward 12 Precinct 15', 'Topeka Ward 12 Precinct 16',
       'Topeka Ward 12 Precinct 81', 'Topeka Ward 13 Precinct 01',
       'Whitfield Voting District', 'Willard Voting District',
       'Topeka Ward 4 Precinct 13', 'Topeka Ward 13 Precinct 06',
       'Topeka Ward 13 Precinct 07', 'Topeka Ward 5 Precinct 13',
       'Stinson Creek Voting District/Ward 2 Precinct 12',
       'Kilmer Voting District', '61st Street Voting District',
       'Montara Voting District', 'Central Pauline Voting District',
       'Topeka Ward 12 Precinct 91', 'Topeka Ward 12 Precinct 93',
       'Topeka Ward 12 Precinct 92', 'Topeka Ward 3 Precinct 10',
       'Topeka Ward 3 Precinct 09', 'Topeka Ward 4 Precinct 14',
       'Topeka Ward 4 Precinct 11', 'Topeka Ward 5 Precinct 01',
       'Topeka Ward 9 Precinct 01', 'Topeka Ward 9 Precinct 12',
       'Wakarusa Voting District', 'Cullen Voting District',
       'Topeka Ward 12 Precinct 14', 'Topeka Ward 12 Precinct 41',
       'Topeka Ward 13 Precinct 02', 'Topeka Ward 13 Precinct 03',
       'Topeka Ward 13 Precinct 22', 'Topeka Ward 13 Precinct 30',
       'Topeka Ward 1 Precinct 08', 'Topeka Ward 10 Precinct 12',
       'Topeka Ward 5 Precinct 15', 'Topeka Ward 5 Precinct 14',
       'Topeka Ward 13 Precinct 12',
       'Topeka Ward 1 Precinct 06/Ward 1 Precinct 07',
       'Topeka Ward 3 Precinct 04', 'Indian Hills Voting District',
       'Topeka Ward 8 Precinct 10', 'Topeka Ward 9 Precinct 08',
       'Topeka Ward 6 Precinct 09', 'Topeka Ward 10 Precinct 05',
       'Topeka Ward 11 Precinct 01', 'Topeka Ward 12 Precinct 03',
       'Topeka Ward 13 Precinct 21', 'Topeka Ward 10 Precinct 11',
       'Topeka Ward 5 Precinct 11', 'Topeka Ward 11 Precinct 02',
       'Topeka Ward 11 Precinct 05', 'Topeka Ward 11 Precinct 07',
       'Topeka Ward 11 Precinct 09', 'Topeka Ward 11 Precinct 03',
       'West Rossville Voting District', 'Cheyenne Voting District',
       'Forbes Voting District', 'Pauline Voting District',
       'Kaw Voting District', 'Topeka Ward 4 Precinct 04',
       'Topeka Ward 4 Precinct 15', 'South Tecumseh Voting District',
       'Little Muddy Creek Voting District',
       'North Mission Voting District', 'Hodges Voting District',
       'Verbena/Lykins Voting District', 'Topeka Ward 13 Precinct 09',
       'Gaillardia Voting District', 'Kingston Voting District',
       'Indianola/Halfday Voting District', 'Topeka Ward 10 Precinct 13',
       'West Wichita Voting District', 'Kiro Voting District',
       'Messhoss Creek Voting District', 'Topeka Ward 14 Precinct 01',
       'Topeka Ward 14 Precinct 03', 'Topeka Ward 4 Precinct 05',
       'East Silver Lake Voting District',
       'City of Auburn Voting District', 'Meadowlark Voting District',
       'Buffalo Voting District', 'Topeka Ward 15 Precinct 01',
       'Iroquois Voting District', 'Apache Voting District',
       'Elmont Voting District', 'Topeka Ward 14 Precinct 05',
       'Topeka Ward 12 Precinct 02', 'Topeka Ward 12 Precinct 20',
       'Topeka Ward 12 Precinct 13', 'Topeka Ward 12 Precinct 31',
       'South Sherwood Voting District', 'East Soldier Voting District',
       'Sherman Voting District', 'Sac Voting District',
       'Fox Voting District', 'Pottawatomie Voting District',
       'Topeka Ward 5 Precinct 10', 'Deer Creek Voting District',
       'Topeka Ward 15 Precinct 03', 'Yucca Voting District',
       'Dover Voting District', 'Topeka Ward 13 Precinct 13',
       'Topeka Ward 10 Precinct 04', 'Topeka Ward 10 Precinct 14',
       'Topeka Ward 13 Precinct 04', 'Central Mission Voting District',
       'Aldersgate Voting District', 'Topeka Ward 13 Precinct 05',
       'Topeka Ward 14 Precinct 04', 'Topeka Ward 14 Precinct 02',
       'West Potter Voting District', 'Topeka Ward 1 Precinct 05',
       'East Wichita Voting District', 'York Outer Voting District',
       'Topeka Ward 6 Precinct 11', 'Topeka Ward 6 Precinct 12',
       'Topeka Ward 6 Precinct 13', 'York Voting District',
       'West Silver Lake Voting District', 'Sunflower Voting District',
       'South Mission Voting District', 'Topeka Ward 1 Precinct 09',
       'Northeast Mission Voting District',
       'North Central Monmouth Voting District',
       'Topeka Ward 12 Precinct 17', 'Vaquero Voting District',
       'Washburn Voting District', 'Ichabod Washburn Voting District',
       'Topeka Ward 13 Precinct 08', 'Topeka Ward 12 Precinct 19',
       'Topeka Ward 13 Precinct 10', 'Topeka Ward 15 Precinct 04',
       'Topeka Ward 15 Precinct 02']

## list1 is from election results and list2 is from per. We want to match each row in election result to a row in pber
list1_s = [
    f"Topeka {entry}" if entry.startswith("Ward") else entry 
    for entry in list1_s
]

list1_s = [re.sub(r'^\d+\s*', '', entry) for entry in list1_s]

list1_s = ['61st Street Voting District' if entry == 'Sixty-First St' else entry for entry in list1_s]
list1_s = [entry for entry in list1_s if not any(keyword in entry for keyword in ['Channel', 'Coup', 'Croco','Flathead', 'Halfday', 'Lykins', 'North Gaillardia', 'SE Kiro', 'Topeka Ward 1 Precinct 6', 'Topeka Ward 1 Precinct 10', 'Topeka Ward 1 Precinct 11', 'Topeka Ward 1 Precinct 12', 'Topeka Ward 1 Precinct 13', 'Topeka Ward 1 Precinct 14', 'Topeka Ward 2 Precinct 12', 'Topeka Ward 10 Precinct 15', 'Topeka Ward 11 Precinct 11' ])]
from natsort import natsorted


sorted_list1_s = natsorted(list1_s)
sorted_list2_s = natsorted(list2_s)

len_list1 = len(sorted_list1_s)
len_list2 = len(sorted_list2_s)

if len_list1 < len_list2:
    sorted_list1_s += [''] * (len_list2 - len_list1)
elif len_list2 < len_list1:
    sorted_list2_s += [''] * (len_list1 - len_list2)


df_comparison_s = pd.DataFrame({
    'List 1': sorted_list1_s,
    'List 2': sorted_list2_s
})
pd.set_option('display.max_rows', None)

df_comparison_s

,List 1,List 2
0,61st Street Voting District,61st Street Voting District
1,Aldersgate,Aldersgate Voting District
2,Apache,Apache Voting District
3,Buffalo,Buffalo Voting District
4,Central Mission,Central Mission Voting District
5,Central Pauline,Central Pauline Voting District
6,Cheyenne,Cheyenne Voting District
7,City Of Auburn,City of Auburn Voting District
8,Cullen,Cullen Voting District
9,Deer Creek,Deer Creek Voting District


In [81]:
replacement_dict_s = dict(zip(sorted_list1_s, sorted_list2_s))
replacement_dict_s = {key.replace('Topeka', '').strip(): value for key, value in replacement_dict_s.items()}

replacement_dict_s['NC Monmouth'] = 'North Central Monmouth Voting District'
replacement_dict_s['NE Monmouth'] = 'Northeast Monmouth Voting District'
replacement_dict_s['NW Monmouth'] = 'Northwest Monmouth Voting District'
replacement_dict_s['North Mission'] = 'North Mission Voting District'
replacement_dict_s['North Mission'] = 'North Mission Voting District'
replacement_dict_s['North Potter'] = 'North Potter Voting District'
replacement_dict_s['North Tecumseh'] = 'North Tecumseh Voting District'
replacement_dict_s['61 Sixty-First St'] = '61st Street Voting Distric'
replacement_dict_s['Topeka Ward 1 Precinct 7'] = 'Topeka Ward 1 Precinct 06/Ward 1 Precinct 07'

precincts_with_numbers = [
    '1 East Rossville', '2 West Rossville', '9 East Silver Lake',
    '10 West Silver Lake', '12 Grove', '14 Kiro', '15 Messhoss Creek',
    '16 SE Kiro', '17 Croco', '18 Flathead', '22 Little Muddy Creek',
    '23 Coup', '24 Sac', '25 Sherman', '26 Iroquois', '27 Halfday',
    '28 West Wichita', '29 Indianola', '30 Apache', '31 East Wichita',
    '32 Fox', '33 Elmont', '34 Kilmer', '35 Rochester',
    '36 East Soldier', '37 Whitfield', '38 Sioux', '39 Pottawatomie',
    '40 Stinson Creek', '41 North Tecumseh', '42 South Tecumseh',
    '43 Pawnee', '44 Kaw', '45 Peck', '46 East Peck', '47 Kiowa',
    '48 Cheyenne', '49 Ponca', '50 North Potter', '51 Channel',
    '52 Deer Creek', '53 Pauline', '54 West Potter',
    '55 Central Pauline', '56 NW Monmouth', '57 NE Monmouth',
    '58 South Monmouth', '59 NC Monmouth', '61 Sixty-First St',
    '62 Forbes', '63 Montara', '64 Cullen', '65 Wakarusa',
    '66 Kingston', '72 City Of Auburn', '73 Meadowlark', '74 Buffalo',
    '78 Hodges Rd', '79 North Gaillardia', '80 Gaillardia', '81 Dover',
    '82 Willard', '83 Ichabod Washburn', '84 Yucca', '85 Aldersgate',
    '86 Sunflower', '87 Washburn', '88 Lykins', '89 North Mission',
       '90 South Mission', '91 Central Mission', '92 Sherwood',
       '93 South Sherwood', '94 Indian Hills', '95 York', '96 Vaquero',
       '97 York Outer', '98 NE Mission', '99 Verbena']

new_replacement_dict_s = {}

for key in replacement_dict_s:
    for precinct_with_number in precincts_with_numbers:
        if precinct_with_number.split(' ', 1)[1] == key:
            new_replacement_dict_s[precinct_with_number] = replacement_dict_s[key]
            break
    else:
        new_replacement_dict_s[key] = replacement_dict_s[key]

for key, value in new_replacement_dict_s.items():
    print(f"'{key}': '{value}',")

# replacement_dict_s

'61st Street Voting District': '61st Street Voting District',
'85 Aldersgate': 'Aldersgate Voting District',
'30 Apache': 'Apache Voting District',
'74 Buffalo': 'Buffalo Voting District',
'91 Central Mission': 'Central Mission Voting District',
'55 Central Pauline': 'Central Pauline Voting District',
'48 Cheyenne': 'Cheyenne Voting District',
'72 City Of Auburn': 'City of Auburn Voting District',
'64 Cullen': 'Cullen Voting District',
'52 Deer Creek': 'Deer Creek Voting District',
'81 Dover': 'Dover Voting District',
'46 East Peck': 'East Peck Voting District',
'1 East Rossville': 'East Rossville Voting District',
'9 East Silver Lake': 'East Silver Lake Voting District',
'36 East Soldier': 'East Soldier Voting District',
'31 East Wichita': 'East Wichita Voting District',
'33 Elmont': 'Elmont Voting District',
'62 Forbes': 'Forbes Voting District',
'32 Fox': 'Fox Voting District',
'80 Gaillardia': 'Gaillardia Voting District',
'12 Grove': 'Grove Voting District',
'78 Hodges Rd': 'Hodge

Use the replacement dict to replace precinct names in election results with those in PBER

In [82]:
SHAWNEE_COUNTY['PRECINCT NAME'] = SHAWNEE_COUNTY['PRECINCT NAME'].replace(new_replacement_dict_s)
len(SHAWNEE_COUNTY)
SHAWNEE_COUNTY.head()

,PRECINCT CODE,PRECINCT NAME,For State Representative 50th District,For State Representative 50th District.1,For State Representative 52nd District,For State Representative 52nd District.1,For State Representative 52nd District.2,For State Representative 53rd District,For State Representative 53rd District.1,For State Representative 53rd District.2,...,For State Representative 54th District.2,For State Representative 55th District,For State Representative 55th District.1,For State Representative 55th District.2,For State Representative 56th District,For State Representative 56th District.1,For State Representative 57th District,For State Representative 57th District.1,For State Representative 58th District,For State Representative 58th District.1
0,NaN,NaN,REP Fred C. Patton,Write-in,DEM Derik Flerlage,REP Jesse Borjon,Write-in,DEM Kirk R. Haskins,REP Bruce Williamson,Write-in,...,Write-in,DEM Tobias Schlingensiepen,REP Todd C. Staerkel,Write-in,DEM Virgil Weigel,Write-in,DEM John Alcala,Write-in,DEM Vic Miller,Write-in
1,1,East Rossville Voting District,324,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,West Rossville Voting District,282,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,East Silver Lake Voting District,432,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,West Silver Lake Voting District,295,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
candidate_party_info = SHAWNEE_COUNTY.iloc[0, 2:] 

df_with_party_cleaned = SHAWNEE_COUNTY.iloc[1:].reset_index(drop=True)

def split_party_candidate(column_name):
    if 'Write-in' in column_name:
        return 'Other', 'Write-in'
    
    parts = column_name.split(' ', 1)
    return parts[0], parts[1] if len(parts) > 1 else ''  # Split into party and candidate


results = []
for idx, row in df_with_party_cleaned.iterrows():
    precinct_code = row['PRECINCT CODE']
    precinct_name = row['PRECINCT NAME']
    
    for col in candidate_party_info.index:
        if row[col] > 0:
            party, candidate = split_party_candidate(candidate_party_info[col])
            votes = row[col]
            race = col.split('.')[0] 
            
            results.append({
                'County': 'Shawnee',
                'Precinct': precinct_name,
                'Race': race,
                'Candidate': candidate,
                'Party': party,
                'Votes': votes,
                'VTD': precinct_code
            })

results_with_party_df_shawnee = pd.DataFrame(results)

results_with_party_df_shawnee.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Shawnee,East Rossville Voting District,For State Representative 50th District,Fred C. Patton,REP,324,1
1,Shawnee,East Rossville Voting District,For State Representative 50th District,Write-in,Other,2,1
2,Shawnee,West Rossville Voting District,For State Representative 50th District,Fred C. Patton,REP,282,2
3,Shawnee,West Rossville Voting District,For State Representative 50th District,Write-in,Other,4,2
4,Shawnee,East Silver Lake Voting District,For State Representative 50th District,Fred C. Patton,REP,432,9


In [84]:
pd.set_option('display.max_rows', None)
House_results_Shawnee = results_with_party_df_shawnee[['County', 'Precinct', 'Race', 'Candidate', 'Party', 'Votes', 'VTD']]
House_results_Shawnee.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Shawnee,East Rossville Voting District,For State Representative 50th District,Fred C. Patton,REP,324,1
1,Shawnee,East Rossville Voting District,For State Representative 50th District,Write-in,Other,2,1
2,Shawnee,West Rossville Voting District,For State Representative 50th District,Fred C. Patton,REP,282,2
3,Shawnee,West Rossville Voting District,For State Representative 50th District,Write-in,Other,4,2
4,Shawnee,East Silver Lake Voting District,For State Representative 50th District,Fred C. Patton,REP,432,9


In [85]:
county_totals_df_Shawnee = House_results_Shawnee[House_results_Shawnee['Precinct'] == 'COUNTY TOTALS']

House_results_Shawnee = House_results_Shawnee[House_results_Shawnee['Precinct'] != 'COUNTY TOTALS']
House_results_Shawnee.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Shawnee,East Rossville Voting District,For State Representative 50th District,Fred C. Patton,REP,324,1
1,Shawnee,East Rossville Voting District,For State Representative 50th District,Write-in,Other,2,1
2,Shawnee,West Rossville Voting District,For State Representative 50th District,Fred C. Patton,REP,282,2
3,Shawnee,West Rossville Voting District,For State Representative 50th District,Write-in,Other,4,2
4,Shawnee,East Silver Lake Voting District,For State Representative 50th District,Fred C. Patton,REP,432,9


In [86]:
len(House_results_Shawnee)

470

In [87]:
len(Shawnee_PBER)

248

### Process WYANDOTTE COUNTY

In [88]:
WYANDOTTE_COUNTY.head(5)

,PRECINCT CODE,PRECINCT NAME,For State Representative 31st District,For State Representative 31st District.1,For State Representative 31st District.2,For State Representative 32nd District,For State Representative 32nd District.1,For State Representative 33rd District,For State Representative 33rd District.1,For State Representative 33rd District.2,...,For State Representative 35th District.1,For State Representative 35th District.2,For State Representative 36th District,For State Representative 36th District.1,For State Representative 36th District.2,For State Representative 37th District,For State Representative 37th District.1,For State Representative 37th District.2,For State Representative 38th District,For State Representative 38th District.1
0,NaN,NaN,REP Dennis Grindel,DEM Louis E. Ruiz,Write-in,DEM Pam Curtis,Write-in,LIB Stephanie Barton,DEM Bill Hutton,REP Mike Thompson,...,REP Sam Stillwell,Write-in,REP Kevin Braun,DEM Lynn Melton,Write-in,DEM Melissa Oropeza,REP Diana Whittington,Write-in,REP Timothy H. Johnson,Write-in
1,1,BS01-01,0,0,0,0,0,9,228,340,...,0,0,0,0,0,0,0,0,0,0
2,2,BS02-01,0,0,0,0,0,6,159,268,...,0,0,0,0,0,0,0,0,0,0
3,3,BS03-01,0,0,0,0,0,19,268,321,...,0,0,0,0,0,0,0,0,0,0
4,4,BS04-01,0,0,0,0,0,19,413,446,...,0,0,0,0,0,0,0,0,0,0


In [89]:
WYANDOTTE_COUNTY['PRECINCT NAME'].unique()

array([nan, 'BS01-01', 'BS02-01', 'BS03-01', 'BS04-01', 'DW01-01',
       'ED01-01', 'ED02-01', 'ED02-02', 'KC01-01', 'KC01-02', 'KC01-03',
       'KC01-04', 'KC01-05', 'KC02-01', 'KC02-02', 'KC02-03', 'KC02-04',
       'KC02-05', 'KC02-06', 'KC02-07', 'KC03-01', 'KC03-02', 'KC03-03',
       'KC03-04', 'KC03-05', 'KC04-01', 'KC04-02', 'KC04-03', 'KC04-04',
       'KC05-01', 'KC05-02', 'KC05-03', 'KC05-04', 'KC05-05', 'KC06-01',
       'KC06-02', 'KC07-01', 'KC07-02', 'KC07-03', 'KC07-04', 'KC07-05',
       'KC07-06', 'KC07-07', 'KC07-08', 'KC07-09', 'KC07-10', 'KC07-11',
       'KC07-12', 'KC08-01', 'KC08-02', 'KC08-03', 'KC08-04', 'KC09-01',
       'KC09-02', 'KC09-03', 'KC09-04', 'KC09-05', 'KC09-06', 'KC09-07',
       'KC09-08', 'KC09-09', 'KC09-10', 'KC09-11', 'KC09-12', 'KC09-13',
       'KC09-14', 'KC09-15', 'KC09-16', 'KC09-17', 'KC09-18', 'KC09-19',
       'KC09-20', 'KC09-21', 'KC09-22', 'KC09-23', 'KC09-24', 'KC09-25',
       'KC10-01', 'KC10-02', 'KC10-03', 'KC10-04', 'KC10-

In [90]:
Wyandotte_PBER = gdf_kansan_2022[gdf_kansan_2022['COUNTYNA'] == 'Wyandotte']
Wyandotte_PBER.head(1)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,...,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA
2461,209,120100,Kansas City Ward 12 Precinct 11,106,125,100,8,339,93,62,...,68,63,73,57,67,60,86,41,"POLYGON Z ((-94.80093 39.04351 0.00000, -94.80...",Wyandotte


In [91]:
Wyandotte_PBER['NAMELSAD'].unique()

array(['Kansas City Ward 12 Precinct 11', 'Edwardsville Precinct 1',
       'Edwardsville Precinct 2', 'Kansas City Ward 14 Precinct 16',
       'Kansas City Ward 6 Precinct 02', 'Kansas City Ward 9 Precinct 01',
       'Kansas City Ward 10 Precinct 01',
       'Bonner Springs Ward 1 Precinct 01',
       'Bonner Springs Ward 2 Precinct 01',
       'Delaware Township 1 Voting District',
       'Bonner Springs Ward 3 Precinct 01',
       'Bonner Springs Ward 4 Precinct 01',
       'Kansas City Ward 1 Precinct 01', 'Kansas City Ward 1 Precinct 02',
       'Kansas City Ward 3 Precinct 03', 'Kansas City Ward 7 Precinct 05',
       'Kansas City Ward 1 Precinct 03', 'Kansas City Ward 2 Precinct 01',
       'Kansas City Ward 2 Precinct 02', 'Kansas City Ward 7 Precinct 04',
       'Kansas City Ward 9 Precinct 13', 'Kansas City Ward 2 Precinct 03',
       'Kansas City Ward 2 Precinct 04', 'Kansas City Ward 8 Precinct 03',
       'Kansas City Ward 9 Precinct 08', 'Kansas City Ward 2 Precinct 05'

In [92]:
list1_wyandotte = ['BS01-01', 'BS02-01', 'BS03-01', 'BS04-01', 'DW01-01',
       'ED01-01', 'ED02-01', 'ED02-02', 'KC01-01', 'KC01-02', 'KC01-03',
       'KC01-04', 'KC01-05', 'KC02-01', 'KC02-02', 'KC02-03', 'KC02-04',
       'KC02-05', 'KC02-06', 'KC02-07', 'KC03-01', 'KC03-02', 'KC03-03',
       'KC03-04', 'KC03-05', 'KC04-01', 'KC04-02', 'KC04-03', 'KC04-04',
       'KC05-01', 'KC05-02', 'KC05-03', 'KC05-04', 'KC05-05', 'KC06-01',
       'KC06-02', 'KC07-01', 'KC07-02', 'KC07-03', 'KC07-04', 'KC07-05',
       'KC07-06', 'KC07-07', 'KC07-08', 'KC07-09', 'KC07-10', 'KC07-11',
       'KC07-12', 'KC08-01', 'KC08-02', 'KC08-03', 'KC08-04', 'KC09-01',
       'KC09-02', 'KC09-03', 'KC09-04', 'KC09-05', 'KC09-06', 'KC09-07',
       'KC09-08', 'KC09-09', 'KC09-10', 'KC09-11', 'KC09-12', 'KC09-13',
       'KC09-14', 'KC09-15', 'KC09-16', 'KC09-17', 'KC09-18', 'KC09-19',
       'KC09-20', 'KC09-21', 'KC09-22', 'KC09-23', 'KC09-24', 'KC09-25',
       'KC10-01', 'KC10-02', 'KC10-03', 'KC10-04', 'KC10-05', 'KC10-06',
       'KC10-07', 'KC11-01', 'KC11-02', 'KC11-03', 'KC11-04', 'KC11-05',
       'KC11-06', 'KC11-07', 'KC11-08', 'KC11-09', 'KC11-10', 'KC11-11',
       'KC11-12', 'KC11-13', 'KC11-14', 'KC11-15', 'KC11-16', 'KC11-17',
       'KC12-01', 'KC12-02', 'KC12-03', 'KC12-04', 'KC12-05', 'KC12-06',
       'KC12-07', 'KC12-08', 'KC12-09', 'KC12-10', 'KC12-11', 'KC12-12',
       'KC12-13', 'KC12-14', 'KC12-15', 'KC12-16', 'KC13-01', 'KC13-02',
       'KC13-03', 'KC13-04', 'KC13-05', 'KC13-06', 'KC13-07', 'KC13-08',
       'KC13-09', 'KC13-10', 'KC13-11', 'KC13-12', 'KC13-13', 'KC14-01',
       'KC14-02', 'KC14-03', 'KC14-04', 'KC14-05', 'KC14-06', 'KC14-07',
       'KC14-08', 'KC14-09', 'KC14-10', 'KC14-11', 'KC14-12', 'KC14-13',
       'KC14-14', 'KC14-15', 'KC14-16', 'KC14-17', 'QC01-01']

list2_wyandotte = ['Kansas City Ward 12 Precinct 11', 'Edwardsville Precinct 1',
       'Edwardsville Precinct 2', 'Kansas City Ward 14 Precinct 16',
       'Kansas City Ward 6 Precinct 02', 'Kansas City Ward 9 Precinct 01',
       'Kansas City Ward 10 Precinct 01',
       'Bonner Springs Ward 1 Precinct 01',
       'Bonner Springs Ward 2 Precinct 01',
       'Delaware Township 1 Voting District',
       'Bonner Springs Ward 3 Precinct 01',
       'Bonner Springs Ward 4 Precinct 01',
       'Kansas City Ward 1 Precinct 01', 'Kansas City Ward 1 Precinct 02',
       'Kansas City Ward 3 Precinct 03', 'Kansas City Ward 7 Precinct 05',
       'Kansas City Ward 1 Precinct 03', 'Kansas City Ward 2 Precinct 01',
       'Kansas City Ward 2 Precinct 02', 'Kansas City Ward 7 Precinct 04',
       'Kansas City Ward 9 Precinct 13', 'Kansas City Ward 2 Precinct 03',
       'Kansas City Ward 2 Precinct 04', 'Kansas City Ward 8 Precinct 03',
       'Kansas City Ward 9 Precinct 08', 'Kansas City Ward 2 Precinct 05',
       'Kansas City Ward 8 Precinct 01', 'Kansas City Ward 8 Precinct 02',
       'Kansas City Ward 3 Precinct 01', 'Kansas City Ward 3 Precinct 04',
       'Kansas City Ward 4 Precinct 02', 'Kansas City Ward 3 Precinct 02',
       'Kansas City Ward 5 Precinct 02', 'Kansas City Ward 4 Precinct 01',
       'Kansas City Ward 5 Precinct 05', 'Kansas City Ward 4 Precinct 03',
       'Kansas City Ward 4 Precinct 04', 'Kansas City Ward 5 Precinct 04',
       'Kansas City Ward 6 Precinct 01', 'Kansas City Ward 5 Precinct 01',
       'Kansas City Ward 8 Precinct 04', 'Kansas City Ward 9 Precinct 02',
       'Kansas City Ward 5 Precinct 03', 'Kansas City Ward 7 Precinct 02',
       'Kansas City Ward 7 Precinct 07', 'Kansas City Ward 7 Precinct 01',
       'Kansas City Ward 7 Precinct 03', 'Kansas City Ward 9 Precinct 03',
       'Kansas City Ward 7 Precinct 06', 'Kansas City Ward 9 Precinct 11',
       'Kansas City Ward 7 Precinct 10', 'Kansas City Ward 7 Precinct 09',
       'Kansas City Ward 9 Precinct 04', 'Kansas City Ward 9 Precinct 05',
       'Kansas City Ward 9 Precinct 07', 'Kansas City Ward 9 Precinct 06',
       'Kansas City Ward 9 Precinct 09', 'Kansas City Ward 9 Precinct 10',
       'Kansas City Ward 9 Precinct 12', 'Kansas City Ward 9 Precinct 14',
       'Kansas City Ward 10 Precinct 02',
       'Kansas City Ward 10 Precinct 03',
       'Kansas City Ward 10 Precinct 04',
       'Kansas City Ward 10 Precinct 05',
       'Kansas City Ward 11 Precinct 01',
       'Kansas City Ward 11 Precinct 02',
       'Kansas City Ward 11 Precinct 03',
       'Kansas City Ward 11 Precinct 04',
       'Kansas City Ward 11 Precinct 05',
       'Kansas City Ward 11 Precinct 07',
       'Kansas City Ward 11 Precinct 08',
       'Kansas City Ward 11 Precinct 10',
       'Kansas City Ward 11 Precinct 11',
       'Kansas City Ward 11 Precinct 12',
       'Kansas City Ward 12 Precinct 01',
       'Kansas City Ward 12 Precinct 03',
       'Kansas City Ward 12 Precinct 05',
       'Kansas City Ward 12 Precinct 07',
       'Kansas City Ward 12 Precinct 08',
       'Kansas City Ward 12 Precinct 09',
       'Kansas City Ward 12 Precinct 10',
       'Kansas City Ward 13 Precinct 02',
       'Kansas City Ward 13 Precinct 03',
       'Kansas City Ward 13 Precinct 04',
       'Kansas City Ward 13 Precinct 07',
       'Kansas City Ward 13 Precinct 08',
       'Kansas City Ward 13 Precinct 09',
       'Kansas City Ward 14 Precinct 01',
       'Kansas City Ward 14 Precinct 02',
       'Kansas City Ward 14 Precinct 06',
       'Kansas City Ward 14 Precinct 07',
       'Kansas City Ward 14 Precinct 08',
       'Kansas City Ward 14 Precinct 09',
       'Kansas City Ward 14 Precinct 10',
       'Kansas City Ward 14 Precinct 12',
       'Kansas City Ward 11 Precinct 06',
       'Kansas City Ward 11 Precinct 15',
       'Kansas City Ward 13 Precinct 01',
       'Kansas City Ward 9 Precinct 16',
       'Kansas City Ward 12 Precinct 02',
       'Kansas City Ward 12 Precinct 06',
       'Kansas City Ward 12 Precinct 04',
       'Kansas City Ward 14 Precinct 11',
       'Kansas City Ward 13 Precinct 05',
       'Kansas City Ward 13 Precinct 06',
       'Kansas City Ward 14 Precinct 03',
       'Kansas City Ward 14 Precinct 04',
       'Kansas City Ward 14 Precinct 05',
       'Kansas City Ward 14 Precinct 13',
       'Kansas City Ward 14 Precinct 14',
       'Lake Quivira City 1 Voting District',
       'Kansas City Ward 9 Precinct 15',
       'Kansas City Ward 14 Precinct 15', 'Edwardsville Precinct 2-2',
       'Kansas City Ward 2 Precinct 06', 'Kansas City Ward 2 Precinct 07',
       'Kansas City Ward 3 Precinct 05', 'Kansas City Ward 7 Precinct 08',
       'Kansas City Ward 7 Precinct 11', 'Kansas City Ward 9 Precinct 24',
       'Kansas City Ward 9 Precinct 19', 'Kansas City Ward 9 Precinct 18',
       'Kansas City Ward 9 Precinct 25', 'Kansas City Ward 9 Precinct 22',
       'Kansas City Ward 9 Precinct 23', 'Kansas City Ward 9 Precinct 17',
       'Kansas City Ward 9 Precinct 21',
       'Kansas City Ward 10 Precinct 06',
       'Kansas City Ward 10 Precinct 07',
       'Kansas City Ward 11 Precinct 17',
       'Kansas City Ward 11 Precinct 16',
       'Kansas City Ward 11 Precinct 09',
       'Kansas City Ward 11 Precinct 14',
       'Kansas City Ward 11 Precinct 13',
       'Kansas City Ward 12 Precinct 15',
       'Kansas City Ward 12 Precinct 16',
       'Kansas City Ward 12 Precinct 14',
       'Kansas City Ward 12 Precinct 13',
       'Kansas City Ward 12 Precinct 12',
       'Kansas City Ward 13 Precinct 13',
       'Kansas City Ward 13 Precinct 10',
       'Kansas City Ward 13 Precinct 11',
       'Kansas City Ward 13 Precinct 12',
       'Kansas City Ward 14 Precinct 17']

from natsort import natsorted

list2_wyandotte.append("Kansas City Ward 1 Precinct 04")
list2_wyandotte.append("Kansas City Ward 1 Precinct 05")
list2_wyandotte.append("Kansas City Ward 7 Precinct 11")
list2_wyandotte.append("Kansas City Ward 9 Precinct 21")


sorted_list1_w = natsorted(list1_wyandotte)
sorted_list2_w = natsorted(list2_wyandotte)

len_list1 = len(sorted_list1_w)
len_list2 = len(sorted_list2_w)

if len_list1 < len_list2:
    sorted_list1_w += [''] * (len_list2 - len_list1)
elif len_list2 < len_list1:
    sorted_list2_w += [''] * (len_list1 - len_list2)

df_comparison_2 = pd.DataFrame({
    'List 1': sorted_list1_w,
    'List 2': sorted_list2_w
})

pd.set_option('display.max_rows', None)
print(df_comparison_2)

df_comparison_2 = pd.DataFrame({
    'List 1': sorted_list1_w,
    'List 2': sorted_list2_w
})
pd.set_option('display.max_rows', None)

df_comparison_2.head()

      List 1                               List 2
0    BS01-01    Bonner Springs Ward 1 Precinct 01
1    BS02-01    Bonner Springs Ward 2 Precinct 01
2    BS03-01    Bonner Springs Ward 3 Precinct 01
3    BS04-01    Bonner Springs Ward 4 Precinct 01
4    DW01-01  Delaware Township 1 Voting District
5    ED01-01              Edwardsville Precinct 1
6    ED02-01              Edwardsville Precinct 2
7    ED02-02            Edwardsville Precinct 2-2
8    KC01-01       Kansas City Ward 1 Precinct 01
9    KC01-02       Kansas City Ward 1 Precinct 02
10   KC01-03       Kansas City Ward 1 Precinct 03
11   KC01-04       Kansas City Ward 1 Precinct 04
12   KC01-05       Kansas City Ward 1 Precinct 05
13   KC02-01       Kansas City Ward 2 Precinct 01
14   KC02-02       Kansas City Ward 2 Precinct 02
15   KC02-03       Kansas City Ward 2 Precinct 03
16   KC02-04       Kansas City Ward 2 Precinct 04
17   KC02-05       Kansas City Ward 2 Precinct 05
18   KC02-06       Kansas City Ward 2 Precinct 06


,List 1,List 2
0,BS01-01,Bonner Springs Ward 1 Precinct 01
1,BS02-01,Bonner Springs Ward 2 Precinct 01
2,BS03-01,Bonner Springs Ward 3 Precinct 01
3,BS04-01,Bonner Springs Ward 4 Precinct 01
4,DW01-01,Delaware Township 1 Voting District


In [93]:
replacement_dict_w = dict(zip(sorted_list1_w, sorted_list2_w))

replacement_dict_w
WYANDOTTE_COUNTY['PRECINCT NAME'] = WYANDOTTE_COUNTY['PRECINCT NAME'].replace(replacement_dict_w)
WYANDOTTE_COUNTY.head()


,PRECINCT CODE,PRECINCT NAME,For State Representative 31st District,For State Representative 31st District.1,For State Representative 31st District.2,For State Representative 32nd District,For State Representative 32nd District.1,For State Representative 33rd District,For State Representative 33rd District.1,For State Representative 33rd District.2,...,For State Representative 35th District.1,For State Representative 35th District.2,For State Representative 36th District,For State Representative 36th District.1,For State Representative 36th District.2,For State Representative 37th District,For State Representative 37th District.1,For State Representative 37th District.2,For State Representative 38th District,For State Representative 38th District.1
0,NaN,NaN,REP Dennis Grindel,DEM Louis E. Ruiz,Write-in,DEM Pam Curtis,Write-in,LIB Stephanie Barton,DEM Bill Hutton,REP Mike Thompson,...,REP Sam Stillwell,Write-in,REP Kevin Braun,DEM Lynn Melton,Write-in,DEM Melissa Oropeza,REP Diana Whittington,Write-in,REP Timothy H. Johnson,Write-in
1,1,Bonner Springs Ward 1 Precinct 01,0,0,0,0,0,9,228,340,...,0,0,0,0,0,0,0,0,0,0
2,2,Bonner Springs Ward 2 Precinct 01,0,0,0,0,0,6,159,268,...,0,0,0,0,0,0,0,0,0,0
3,3,Bonner Springs Ward 3 Precinct 01,0,0,0,0,0,19,268,321,...,0,0,0,0,0,0,0,0,0,0
4,4,Bonner Springs Ward 4 Precinct 01,0,0,0,0,0,19,413,446,...,0,0,0,0,0,0,0,0,0,0


In [94]:
candidate_party_info = WYANDOTTE_COUNTY.iloc[0, 2:] 

df_with_party_cleaned = WYANDOTTE_COUNTY.iloc[1:].reset_index(drop=True)

def split_party_candidate(column_name):
    if 'Write-in' in column_name:
        return 'Other', 'Write-in'
    
    parts = column_name.split(' ', 1)
    return parts[0], parts[1] if len(parts) > 1 else ''  # Split into party and candidate


results = []
for idx, row in df_with_party_cleaned.iterrows():
    precinct_code = row['PRECINCT CODE']
    precinct_name = row['PRECINCT NAME']
    
    for col in candidate_party_info.index:
        if row[col] > 0:
            party, candidate = split_party_candidate(candidate_party_info[col])
            votes = row[col]
            race = col.split('.')[0] 
            
            results.append({
                'County': 'Wyandotte',
                'Precinct': precinct_name,
                'Race': race,
                'Candidate': candidate,
                'Party': party,
                'Votes': votes,
                'VTD': precinct_code
            })

results_with_party_df_wyandotte = pd.DataFrame(results)

results_with_party_df_wyandotte.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Stephanie Barton,LIB,9,1
1,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Bill Hutton,DEM,228,1
2,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Mike Thompson,REP,340,1
3,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Write-in,Other,2,1
4,Wyandotte,Bonner Springs Ward 2 Precinct 01,For State Representative 33rd District,Stephanie Barton,LIB,6,2


In [95]:
pd.set_option('display.max_rows', None)
House_results_Wyandotte = results_with_party_df_wyandotte[['County', 'Precinct', 'Race', 'Candidate', 'Party', 'Votes', 'VTD']]
House_results_Wyandotte.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Stephanie Barton,LIB,9,1
1,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Bill Hutton,DEM,228,1
2,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Mike Thompson,REP,340,1
3,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Write-in,Other,2,1
4,Wyandotte,Bonner Springs Ward 2 Precinct 01,For State Representative 33rd District,Stephanie Barton,LIB,6,2


In [96]:
county_totals_df_Wyandotte = House_results_Wyandotte[House_results_Wyandotte['Precinct'] == 'COUNTY TOTALS']

House_results_Wyandotte = House_results_Wyandotte[House_results_Wyandotte['Precinct'] != 'COUNTY TOTALS']
House_results_Wyandotte.head()

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Stephanie Barton,LIB,9,1
1,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Bill Hutton,DEM,228,1
2,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Mike Thompson,REP,340,1
3,Wyandotte,Bonner Springs Ward 1 Precinct 01,For State Representative 33rd District,Write-in,Other,2,1
4,Wyandotte,Bonner Springs Ward 2 Precinct 01,For State Representative 33rd District,Stephanie Barton,LIB,6,2


In [97]:
len(House_results_Wyandotte['Precinct'].unique())

143

In [98]:
House_Results_4Counties = pd.concat([House_results_Sedgwick, results_df_JOHNSON_COUNTY, House_results_Wyandotte, House_results_Shawnee], axis=0, ignore_index=True)
len(House_Results_4Counties)
House_Results_4Counties.head()
House_Results_4Counties['County'].unique()

array(['Sedgwick', 'Johnson', 'Wyandotte', 'Shawnee'], dtype=object)

### Process the other 101 counties

In [99]:
KSHouse.head(5)

,County,Precinct,Race,Candidate,Party,Votes,VTD
0,Allen,Carlyle Township,Kansas House of Representatives 9,"Cloutier, Alana",Democratic,26.0,000010
1,Allen,Carlyle Township,Kansas House of Representatives 9,"Gardner, Fred",Republican,101.0,000010
2,Allen,Cottage Grove Township,Kansas House of Representatives 9,"Cloutier, Alana",Democratic,19.0,000020
3,Allen,Cottage Grove Township,Kansas House of Representatives 9,"Gardner, Fred",Republican,67.0,000020
4,Allen,Deer Creek Township,Kansas House of Representatives 9,"Cloutier, Alana",Democratic,10.0,000030


Check how many no vote precincts there are in the provided election results

In [100]:
unique_combinations1 = JOHNSON_COUNTY[['PRECINCT CODE', 'PRECINCT NAME']].drop_duplicates()
unique_combinations2 = SEDGWICK_COUNTY[['PRECINCT CODE', 'PRECINCT NAME']].drop_duplicates()
unique_combinations3 = SHAWNEE_COUNTY[['PRECINCT CODE', 'PRECINCT NAME']].drop_duplicates()
unique_combinations4 = WYANDOTTE_COUNTY[['PRECINCT CODE', 'PRECINCT NAME']].drop_duplicates()

len(unique_combinations1) + len(unique_combinations2) + len(unique_combinations3) + len(unique_combinations4)

1394

Using a similar approach to create column names and pivot

In [101]:
# Function to swap the parts of the name
def swap_name(name):
    # add a try catach
    try:
        parts = name.split(',')
        if len(parts) == 2:
            return parts[1].strip() + ' ' + parts[0].strip()
    except:
        return name
    return name

KSHouse['Candidate'] = KSHouse['Candidate'].str.strip().str.upper()
KSHouse['Candidate'] = KSHouse['Candidate'].apply(swap_name)


In [102]:
KSHouse = pd.concat([KSHouse, House_Results_4Counties], axis=0, ignore_index=True)


### Previous code that creates columns(pivoting)

In [103]:
election_results_cleaned = KSHouse

# Extract district number from the 'Race' column and create new column
election_results_cleaned['District'] = election_results_cleaned['Race'].str.extract(r'(\d+)', expand=False)

# Function to create a unique column name based on the office, district, candidate, and party
def create_column_name(row):
    office_abbr = "SL"  # Abbreviation for State Legislature
    party_initial = row['Party'][0]
    last_name = row['Candidate'].split()[-1][:3].upper()
    district = row['District'].zfill(3)  # Ensure the district number has leading zeros
    
    # Create a unique column name that includes office, district, candidate, and party
    unique_name = f"{office_abbr}{district}{party_initial}{last_name}"
    
    return unique_name

# Apply the function to generate a unique column name for each candidate
election_results_cleaned['ColumnName'] = election_results_cleaned.apply(create_column_name, axis=1)

# Pivot dataset so that each column represents a candidate (unique column name) and each row represents a precinct
pivoted_df = election_results_cleaned.pivot_table(
    index=['County', 'Precinct', 'VTD'],
    columns='ColumnName',
    values='Votes',
    aggfunc='sum'
)

# Flatten the columns
pivoted_df.columns = [col for col in pivoted_df.columns]

# Reset index
pivoted_df.reset_index(inplace=True)

# Display the final DataFrame
pivoted_df.head(5)

,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,...,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA
0,Allen,Bassett City Enclave,00020C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allen,Carlyle Township,000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Allen,Cottage Grove Township,000020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Allen,Deer Creek Township,000030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Allen,East Elm,000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Both of these files have 0 no vote districts. When merge, make sure all precincts in election results have a corresponding precinct in PBER.

# Merge Kansas House election results with PBER from IKE Lab

#### Merge 4 counites first, as they have different format than the rest of the counties. Have to manually process county by county because the given VTD/precinct name can be very different 

In [104]:
pivoted_df_4 = pivoted_df[pivoted_df['County'].isin(['Sedgwick', 'Johnson', 'Wyandotte', 'Shawnee'])]

pivoted_df_4['County'].unique()

array(['Johnson', 'Sedgwick', 'Shawnee', 'Wyandotte'], dtype=object)

### Merge Sedgwick county

In [105]:
Sedgwick_PBER.head(3)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,...,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA
3850,173,000000,101,632,465,841,29,1967,296,342,...,363,173,355,186,376,165,368,168,"POLYGON Z ((-97.29881 37.66457 0.00000, -97.30...",Sedgwick
3851,173,000010,102,364,552,306,19,1241,367,360,...,429,181,441,178,442,175,455,157,"POLYGON Z ((-97.26217 37.69016 0.00000, -97.26...",Sedgwick
3852,173,000020,103,112,97,111,9,329,43,81,...,90,20,87,24,91,22,74,37,"POLYGON Z ((-97.32639 37.67871 0.00000, -97.32...",Sedgwick


In [106]:
pivoted_sedgwick = pivoted_df[pivoted_df['County'] == 'Sedgwick']
pivoted_sedgwick.head()

,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,...,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA
3363,Sedgwick,101,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3364,Sedgwick,102,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3365,Sedgwick,103,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3366,Sedgwick,104,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3367,Sedgwick,105,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check the type before merge

In [107]:
print(pivoted_sedgwick['Precinct'].apply(type).value_counts())
print(Sedgwick_PBER['NAMELSAD'].apply(type).value_counts())

Precinct
<class 'int'>    194
<class 'str'>    170
Name: count, dtype: int64
NAMELSAD
<class 'str'>    518
Name: count, dtype: int64


Change int to str

In [108]:
pivoted_sedgwick['Precinct'] = pivoted_sedgwick['Precinct'].apply(lambda x: str(x) if isinstance(x, int) else x)
pivoted_sedgwick['Precinct'].apply(type).value_counts()


Precinct
<class 'str'>    364
Name: count, dtype: int64

merge NAMELSAD to Precinct

In [109]:
merged_df_sedgwick = pd.merge(Sedgwick_PBER, pivoted_sedgwick, left_on=['NAMELSAD'], right_on=['Precinct'], how='outer', indicator=True)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
merged_df_sedgwick.head(1)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,...,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
0,173,000000,101,632,465,841,29,1967,296,342,22,407,235,9,9,292,338,20,259,390,262,369,19,314,323,388,167,363,178,363,173,355,186,376,165,368,168,"POLYGON Z ((-97.29881 37.66457 0.00000, -97.30...",Sedgwick,Sedgwick,...,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [110]:
merged_df_sedgwick['_merge'].value_counts()

_merge
both          364
left_only     154
right_only      0
Name: count, dtype: int64

### Merge Johnson county

In [111]:
pivoted_Johnson = pivoted_df[pivoted_df['County'] == 'Johnson']
len(pivoted_Johnson)


541

In [112]:
merged_df_Johnson = pd.merge(Johnson_PBER, pivoted_Johnson, left_on=['NAMELSAD'], right_on=['Precinct'], how='outer', indicator=True)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
merged_df_Johnson.head(1)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,...,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
0,091,000000,Aubry Twp 0-01,40,97,38,1,176,71,50,1,57,65,1,0,69,50,4,66,58,68,47,7,70,51,77,37,73,39,78,36,78,36,76,37,82,30,"POLYGON Z ((-94.67206 38.78189 0.00000, -94.67...",Johnson,Johnson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [113]:
merged_df_Johnson['_merge'].value_counts()

_merge
both          541
left_only      69
right_only      0
Name: count, dtype: int64

### Merge Wyandotte county

In [114]:
pivoted_Wyandotte = pivoted_df[pivoted_df['County'] == 'Wyandotte']
pivoted_Wyandotte.head(5)
# len(pivoted_Wyandotte)

,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,...,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA
4269,Wyandotte,Bonner Springs Ward 1 Precinct 01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4270,Wyandotte,Bonner Springs Ward 2 Precinct 01,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4271,Wyandotte,Bonner Springs Ward 3 Precinct 01,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4272,Wyandotte,Bonner Springs Ward 4 Precinct 01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4273,Wyandotte,Delaware Township 1 Voting District,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
Wyandotte_PBER.head(5)
# len(Wyandotte_PBER)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA
2461,209,120100,Kansas City Ward 12 Precinct 11,106,125,100,8,339,93,62,6,71,86,5,3,91,63,7,89,75,90,62,7,95,59,76,59,71,61,68,63,73,57,67,60,86,41,"POLYGON Z ((-94.80093 39.04351 0.00000, -94.80...",Wyandotte
2462,209,140020,Edwardsville Precinct 1,788,829,902,33,2552,588,434,39,533,500,26,14,573,444,30,532,531,534,427,80,590,429,526,321,512,318,505,320,503,318,496,319,559,254,"POLYGON Z ((-94.85361 39.10375 0.00000, -94.85...",Wyandotte
2463,209,140030,Edwardsville Precinct 2,146,125,132,6,409,91,71,8,83,79,5,4,85,70,13,80,90,87,71,9,94,71,78,51,69,60,70,57,74,52,74,53,86,44,"POLYGON Z ((-94.80529 39.10282 0.00000, -94.80...",Wyandotte
2464,209,140090,Kansas City Ward 14 Precinct 16,243,128,83,2,456,116,153,8,180,94,3,2,122,148,6,97,179,99,162,10,124,144,177,66,174,64,173,67,168,67,169,61,185,46,"POLYGON Z ((-94.81860 39.13577 0.00000, -94.81...",Wyandotte
2465,209,600310,Kansas City Ward 6 Precinct 02,89,35,101,1,226,18,19,1,24,14,0,1,18,19,2,14,24,16,20,3,20,18,22,7,20,9,18,11,18,9,22,5,22,5,"POLYGON Z ((-94.67986 39.09997 0.00000, -94.67...",Wyandotte


In [116]:
merged_df_Wyandotte = pd.merge(Wyandotte_PBER, pivoted_Wyandotte, left_on=['NAMELSAD'], right_on=['Precinct'], how='outer', indicator=True)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
merged_df_Wyandotte.head(1)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,...,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
0,209,120100,Kansas City Ward 12 Precinct 11,106,125,100,8,339,93,62,6,71,86,5,3,91,63,7,89,75,90,62,7,95,59,76,59,71,61,68,63,73,57,67,60,86,41,"POLYGON Z ((-94.80093 39.04351 0.00000, -94.80...",Wyandotte,Wyandotte,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [117]:
merged_df_Wyandotte['_merge'].value_counts()

_merge
both          143
left_only       1
right_only      0
Name: count, dtype: int64

In [118]:
pd.set_option('display.max_columns', None)

merged_df_Wyandotte[merged_df_Wyandotte['_merge'] == 'right_only']


,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge


### Merge Shawnee county

In [119]:
pivoted_Shawnee = pivoted_df[pivoted_df['County'] == 'Shawnee']
pivoted_Shawnee.head(5)
# len(pivoted_Shawnee)

,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA
3751,Shawnee,Aldersgate Voting District,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,NaN,113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3752,Shawnee,Apache Voting District,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,392.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [120]:
Shawnee_PBER.head(5)
# len(Shawnee_PBER)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA
1936,177,000080,East Peck Voting District,120,199,119,7,445,170,103,12,159,115,9,3,162,104,12,138,145,141,125,13,208,70,160,95,151,99,151,101,161,91,160,92,175,80,"POLYGON Z ((-95.59846 39.02094 0.00000, -95.59...",Shawnee
1937,177,000090,East Rossville Voting District,135,360,159,7,661,297,81,10,159,214,17,3,265,100,18,238,150,246,110,26,310,71,211,143,200,150,191,156,215,137,211,141,243,112,"POLYGON Z ((-95.96139 39.19042 0.00000, -95.96...",Shawnee
1938,177,000150,Grove Voting District,84,317,143,3,547,262,55,11,121,199,7,1,258,63,6,211,112,232,82,13,274,51,174,111,173,113,170,114,184,106,178,107,201,86,"POLYGON Z ((-95.90557 39.13052 0.00000, -95.90...",Shawnee
1939,177,000660,Topeka Ward 3 Precinct 08,193,92,173,10,468,57,112,6,122,45,5,4,58,111,6,53,122,58,111,7,74,97,106,50,99,58,103,52,108,50,109,50,104,53,"POLYGON Z ((-95.69183 39.03894 0.00000, -95.69...",Shawnee
1940,177,001010,Topeka Ward 6 Precinct 08,286,127,263,5,681,118,139,17,189,76,6,6,99,161,15,85,190,87,170,18,130,143,178,69,172,73,173,71,179,69,183,62,176,71,"POLYGON Z ((-95.68769 38.99205 0.00000, -95.68...",Shawnee


In [121]:
merged_df_Shawnee = pd.merge(Shawnee_PBER, pivoted_Shawnee, left_on=['NAMELSAD'], right_on=['Precinct'], how='outer', indicator=True)
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
merged_df_Shawnee.head(1)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,...,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
0,177,000080,East Peck Voting District,120.0,199.0,119.0,7.0,445.0,170.0,103.0,12.0,159.0,115.0,9.0,3.0,162.0,104.0,12.0,138.0,145.0,141.0,125.0,13.0,208.0,70.0,160.0,95.0,151.0,99.0,151.0,101.0,161.0,91.0,160.0,92.0,175.0,80.0,"POLYGON Z ((-95.59846 39.02094 0.00000, -95.59...",Shawnee,Shawnee,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [122]:
merged_df_Shawnee['_merge'].value_counts()

_merge
both          220
left_only      28
right_only      1
Name: count, dtype: int64

In [123]:
pd.set_option('display.max_columns', None)

merged_df_Shawnee[merged_df_Shawnee['_merge'] == 'right_only']

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,Shawnee,Ward 1 Precinct 6,106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,361.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,right_only


In [124]:
merged_df_Shawnee = merged_df_Shawnee[merged_df_Shawnee['_merge'] != 'right_only']


### Merge other 101 counties

In [125]:
counties_to_remove = ['Sedgwick', 'Johnson', 'Wyandotte', 'Shawnee']

pivoted_df = pivoted_df[~pivoted_df['County'].isin(counties_to_remove)]

In [126]:
merged_df = pd.merge(gdf_kansan_2022, pivoted_df, left_on=['COUNTYNA', 'VTDST'], right_on=['County', 'VTD'], how='left', indicator=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

merged_df.head()

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
0,001,000010,Carlyle Township Voting District,28,165,42,0,235,104,19,3,31,93,0,2,96,26,4,94,32,94,26,6,102,23,71,51,69,53,66,56,67,55,65,57,72,48,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42...",Allen,Allen,Carlyle Township,000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,001,000020,Cottage Grove Township Voting District,19,108,4

Counties with left_only merge are the ones we have to take a deeper look into.

In [127]:
merge_counts_by_county = merged_df.groupby('COUNTYNA')['_merge'].value_counts()

merge_counts_by_county

COUNTYNA      _merge    
Allen         both           26
              left_only       0
              right_only      0
Anderson      both           18
              left_only       0
              right_only      0
Atchison      both           27
              left_only       0
              right_only      0
Barber        both           22
              left_only       0
              right_only      0
Barton        both           41
              left_only      10
              right_only      0
Bourbon       both           28
              left_only       1
              right_only      0
Brown         both           27
              left_only       1
              right_only      0
Butler        both           53
              left_only      14
              right_only      0
Chase         both            9
              left_only       2
              right_only      0
Chautauqua    both           14
              left_only       0
              right_only      0
Cherokee      b

No right_only merge shows that all precincts in PBER have a corresponding senate election result, so it is ok to use left merge

In [128]:
right_only_counties = merge_counts_by_county[(merge_counts_by_county.index.get_level_values('_merge') == 'right_only') & (merge_counts_by_county > 0)]

for county in right_only_counties.index.get_level_values('COUNTYNA'):
    print(county)


For example, in Allen, row 21-25 are no vote precincts in both files

In [129]:
Allen = merged_df[merged_df['COUNTYNA'] == 'Allen']
Allen.head(1)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
0,001,000010,Carlyle Township Voting District,28,165,42,0,235,104,19,3,31,93,0,2,96,26,4,94,32,94,26,6,102,23,71,51,69,53,66,56,67,55,65,57,72,48,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42...",Allen,Allen,Carlyle Township,000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


Same is for Barton. So I believe it is ok to assume that PBER group precincts in a way to create the precinct boundaries.

In [130]:
Barton = merged_df[merged_df['COUNTYNA'] == 'Barton']
Barton.head(1)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
93,009,000010,Albion Township Voting District,5,22,13,0,40,16,3,2,7,13,0,1,16,3,2,13,8,14,5,2,17,4,10,10,11,9,12,8,10,10,10,10,13,7,"POLYGON Z ((-98.92328 38.56675 0.00000, -98.92...",Barton,Barton,Albion Township,000010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [131]:
# Check if two DataFrames have the same columns
if merged_df.columns.equals(merged_df_Johnson.columns):
    print("The DataFrames have the same columns.")
else:
    print("The DataFrames do not have the same columns.")

The DataFrames have the same columns.


### Check that after pivoting, the numbers match previous results(they did)

In [132]:
import pandas as pd

def check_total_votes_after_pivot(df, original_df):
    df.reset_index(inplace=True)

    candidate_totals = {}

    for column in df.columns:
        if column not in ['County', 'Precinct', 'VTD']:
            candidate_totals[column] = df[column].sum()

    original_totals = original_df.groupby('ColumnName')['Votes'].sum()

    pivoted_totals_df = pd.DataFrame(list(candidate_totals.items()), columns=['Candidate', 'TotalVotes_After_Pivot'])
    original_totals_df = original_totals.reset_index().rename(columns={'ColumnName': 'Candidate', 'Votes': 'TotalVotes_Original'})

    comparison_df = pd.merge(pivoted_totals_df, original_totals_df, on='Candidate', how='outer')

    return comparison_df

check_total_votes_after_pivot(pivoted_df, election_results_cleaned)



,Candidate,TotalVotes_After_Pivot,TotalVotes_Original
0,index,6027932.0,NaN
1,SL001DROG,1928.0,1928.0
2,SL001RHOU,5847.0,5847.0
3,SL002RCOL,6566.0,6566.0
4,SL003RSMI,4714.0,4714.0
5,SL004RJAC,7069.0,7069.0
6,SL005OWRI,0.0,43.0
7,SL005RBAR,5705.0,7085.0
8,SL006DFRI,2954.0,2954.0
9,SL006RPAR,6265.0,6265.0


In [133]:
merged_df.tail(5)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
4374,135,000040,Franklin Township Voting District,5,69,8,1,83,54,0,2,8,45,2,1,52,1,3,49,7,52,3,2,53,1,28,19,26,22,26,23,25,23,26,22,32,17,"POLYGON Z ((-100.02656 38.43520 0.00000, -99.9...",Ness,Ness,Franklin Township,000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4375,135,000050,Highpoint Township Voting District,2,27,4,0,33,22,0,1,1

### Clean the merged dataframes

In [134]:
columns_to_remove = ['County', 'Precinct', 'VTD', '_merge']

merged_df = merged_df.drop(columns=columns_to_remove)
merged_df = merged_df.fillna(0)
columns = ['COUNTYNA'] + [col for col in merged_df.columns if col != 'COUNTYNA']
pivoted_df_cleaned = merged_df[columns]

columns_last = [col for col in pivoted_df_cleaned.columns if col != 'geometry'] + ['geometry']

concatenated_df = pivoted_df_cleaned[columns_last]

concatenated_df.head()

,COUNTYNA,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,geometry
0,Allen,001,000010,Carlyle Township Voting District,28,165,42,0,235,104,19,3,31,93,0,2,96,26,4,94,32,94,26,6,102,23,71,51,69,53,66,56,67,55,65,57,72,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42..."
1,Allen,001,000020,Cottage Grove Township Voting District,19,108,43,1,171,72,16,0,18,70,0,0,70,16,1,69,20,69,16,2,71,16,33

In [135]:
dfs = [merged_df_sedgwick, merged_df_Johnson, merged_df_Shawnee, merged_df_Wyandotte]

common_columns = dfs[0].columns

same_columns = all(df.columns.equals(common_columns) for df in dfs)

if same_columns:
    print("All DataFrames have the same columns.")
else:
    print("The DataFrames do not have the same columns.")

merged_df_4_counties = pd.concat(dfs, axis=0, ignore_index=True)


All DataFrames have the same columns.


In [136]:
merged_df_4_counties.head(1)

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
0,173,000000,101,632.0,465.0,841.0,29.0,1967.0,296.0,342.0,22.0,407.0,235.0,9.0,9.0,292.0,338.0,20.0,259.0,390.0,262.0,369.0,19.0,314.0,323.0,388.0,167.0,363.0,178.0,363.0,173.0,355.0,186.0,376.0,165.0,368.0,168.0,"POLYGON Z ((-97.29881 37.66457 0.00000, -97.30...",Sedgwick,Sedgwick,101,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,337.0,292.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [137]:
missing_geometry_rows = merged_df_4_counties[merged_df_4_counties['geometry'].isnull()]

# Print the rows with missing geometry values
if not missing_geometry_rows.empty:
    print("Rows with missing geometry values:")
    print(missing_geometry_rows)
else:
    print("No rows with missing geometry values.")

No rows with missing geometry values.


need to clean the right_only ones

In [138]:
merged_df_4_counties = merged_df_4_counties.dropna(subset=['geometry'])
merged_df_4_counties.head()

,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,geometry,COUNTYNA,County,Precinct,VTD,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,_merge
0,173,000000,101,632.0,465.0,841.0,29.0,1967.0,296.0,342.0,22.0,407.0,235.0,9.0,9.0,292.0,338.0,20.0,259.0,390.0,262.0,369.0,19.0,314.0,323.0,388.0,167.0,363.0,178.0,363.0,173.0,355.0,186.0,376.0,165.0,368.0,168.0,"POLYGON Z ((-97.29881 37.66457 0.00000, -97.30...",Sedgwick,Sedgwick,101,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,337.0,292.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,1

In [139]:
columns_to_remove = ['County', 'Precinct', 'VTD', '_merge']

merged_df_4_counties = merged_df_4_counties.drop(columns=columns_to_remove)

merged_df_4_counties = merged_df_4_counties.fillna(0)

columns = ['COUNTYNA'] + [col for col in merged_df_4_counties.columns if col != 'COUNTYNA']

columns_last = [col for col in columns if col != 'geometry'] + ['geometry']

merged_df_4_counties_cleaned = merged_df_4_counties[columns_last]

merged_df_4_counties_cleaned.head()


,COUNTYNA,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,geometry
0,Sedgwick,173,000000,101,632.0,465.0,841.0,29.0,1967.0,296.0,342.0,22.0,407.0,235.0,9.0,9.0,292.0,338.0,20.0,259.0,390.0,262.0,369.0,19.0,314.0,323.0,388.0,167.0,363.0,178.0,363.0,173.0,355.0,186.0,376.0,165.0,368.0,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,337.0,292.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-97.29881 37.66457 0.00000, -97.30..."
1,Sedgwick,173,000010,102,364.0,552.0,306.0,19.0,1

### Merge 101 counties with 4 counties

In [140]:
if concatenated_df.columns.equals(merged_df_4_counties_cleaned.columns):
    print("The DataFrames have the same columns.")
else:
    print("The DataFrames do not have the same columns.")

if concatenated_df.columns.equals(merged_df_4_counties_cleaned.columns):
    combined_df = pd.concat([concatenated_df, merged_df_4_counties_cleaned], axis=0).reset_index(drop=True)
    print("DataFrames concatenated successfully.")
else:
    print("DataFrames do not have the same columns, cannot concatenate.")


The DataFrames have the same columns.
DataFrames concatenated successfully.


### Merge KS House with KS Senate

In [141]:
pivoted_df_senate = pivoted_df_senate[
    ~((pivoted_df_senate['SU038DLAR'] == 0) & (pivoted_df_senate['SU038RRYC'] == 0))
]

In [142]:
row1 = pivoted_df_senate[pivoted_df_senate['Precinct'] == 'Dodge Township Ward 3 H119 B']
row2 = pivoted_df_senate[pivoted_df_senate['Precinct'] == 'Dodge Township W H119']

if not row1.empty and not row2.empty:
    combined_row = row1.copy()
    combined_row['SU038DLAR'] = row1['SU038DLAR'].values[0] + row2['SU038DLAR'].values[0]
    combined_row['SU038RRYC'] = row1['SU038RRYC'].values[0] + row2['SU038RRYC'].values[0]
    
    combined_row.update(row2.drop(columns=['SU038DLAR', 'SU038RRYC']))
    
    combined_row['Precinct'] = 'Dodge Township Ward 3 119'
    
    pivoted_df_senate = pivoted_df_senate[~pivoted_df_senate['Precinct'].isin(['Dodge Township Ward 3 H119 B', 'Dodge Township W H119'])]

    pivoted_df_senate = pd.concat([pivoted_df_senate, combined_row], ignore_index=True)

print(pivoted_df_senate.head())


  County            Precinct     VTD  SU038DLAR  SU038RRYC
0  Clark   Appleton Township  000010         30        282
1  Clark            Center 1  000020         27        189
2  Clark            Center 2  000030         18        165
3  Clark  Englewood Township  000040         14         50
4  Clark  Lexington Township  000050          0         21


In [143]:
replacement_dict_senate = {
  'Grandview Township Ward 2 H119 C': 'Grandview Township East H115 Voting District',
  'Grandview Township Ward 2 H115 A': 'Grandview Township East H115 Voting District',
  'Dodge Township Ward 3 119': 'Dodge Township Ward 3 119',
  'Dodge Township Ward 3 H115 A': 'Dodge Township Ward 3 115',
  'Dodge Township N H119': 'Dodge Township N H119 Voting District',
  'Dodge Township H115': 'Dodge Township H115 Voting District',
  'Enterprise Township H119': 'Enterprise Township H119 Voting District',
    'Dodge City Precinct 01  H115 A': 'Dodge City Precinct 01 115',
    'Dodge City Precinct 01 H119 B': 'Dodge City Precinct 01 119',
    'Dodge City Precinct 02 H119 B': 'Dodge City Precinct 02',
    'Dodge City Precinct 03 H119 B': 'Dodge City Precinct 03',
    'Dodge City Precinct 04 H115 A': 'Dodge City Precinct 04 115',
    'Dodge City Precinct 04 H119 B': 'Dodge City Precinct 04 119',
    'Dodge City Precinct 06 H115 A': 'Dodge City Precinct 06 115',
    'Dodge City Precinct 06 H119 B': 'Dodge City Precinct 06 119',
    'Dodge City Precinct 09 H119 B': 'Dodge City Precinct 09 B',
}

pivoted_df_senate['Precinct'] = pivoted_df_senate['Precinct'].replace(replacement_dict_senate)

# Create the replacement dictionary
replacement_vtdst_dict = {
    'Dodge City Airport Voting District': '00005B',
    'Dodge City Excel #1 Voting District': '00008B',
    'Dodge City Excel #2 Voting District': '00008C',
    'Dodge City Industrial Park Voting District': '00005C',
    'Dodge City Millard Voting District': '00008D',
    'Dodge City Precinct 01 115': '00005A',
    'Dodge City Precinct 01 119': '10005A',
    'Dodge City Precinct 02': '000060',
    'Dodge City Precinct 03': '000070',
    'Dodge City Precinct 04 115': '00008A',
    'Dodge City Precinct 04 119': '10008A',
    'Dodge City Precinct 06 115': '000100',
    'Dodge City Precinct 06 119': '100100',
    'Dodge City Precinct 08': '800010',
    'Dodge City Precinct 09 B': '800021',
    'Dodge City Precinct 10': '800030',
    'Dodge City Precinct 11': '800040',
    'Dodge Township H115 Voting District': '000190',
    'Dodge Township N H119 Voting District': '000191',
    'Dodge Township Ward 3 115': '000200',
    'Dodge Township Ward 3 119': '000201',
    'Enterprise Township H119 Voting District': '00021A',
    'Grandview Township East H115 Voting District': '00025B',
    'Grandview Township H119 Voting District': '00025A',
    'Grandview Township Ward 2 115': '000290',
    'Grandview Township Ward 2 119 Voting District': '00029A'
}

pivoted_df_senate['VTD'] = pivoted_df_senate['Precinct'].map(replacement_vtdst_dict).fillna(pivoted_df_senate['VTD'])

pivoted_df_senate.head()



,County,Precinct,VTD,SU038DLAR,SU038RRYC
0,Clark,Appleton Township,000010,30,282
1,Clark,Center 1,000020,27,189
2,Clark,Center 2,000030,18,165
3,Clark,Englewood Township,000040,14,50
4,Clark,Lexington Township,000050,0,21


In [144]:
final_df = pd.merge(combined_df, pivoted_df_senate, left_on=['COUNTYNA', 'VTDST'], right_on=['County', 'VTD'], how='outer', indicator=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

final_df.head()



,COUNTYNA,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,geometry,County,Precinct,VTD,SU038DLAR,SU038RRYC,_merge
0,Allen,001,000010,Carlyle Township Voting District,28.0,165.0,42.0,0.0,235.0,104.0,19.0,3.0,31.0,93.0,0.0,2.0,96.0,26.0,4.0,94.0,32.0,94.0,26.0,6.0,102.0,23.0,71.0,51.0,69.0,53.0,66.0,56.0,67.0,55.0,65.0,57.0,72.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42...",NaN,NaN

In [145]:
final_df['_merge'].value_counts()

_merge
left_only     5803
both            97
right_only       0
Name: count, dtype: int64

In [146]:
final_df[final_df['_merge'] == 'right_only']

,COUNTYNA,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,geometry,County,Precinct,VTD,SU038DLAR,SU038RRYC,_merge


In [147]:
final_df['_merge'].value_counts()

_merge
left_only     5803
both            97
right_only       0
Name: count, dtype: int64

In [148]:
columns_to_remove = ['County', 'Precinct', 'VTD', '_merge']

final_df = final_df.drop(columns=columns_to_remove)

final_df = final_df.fillna(0)

columns_last = [col for col in final_df.columns if col != 'geometry'] + ['geometry']

final_df = final_df[columns_last]

final_df.head()


,COUNTYNA,COUNTYFP,VTDST,NAMELSAD,REG_DEM,REG_REP,REG_UNA,REG_LIB,REG_TOTAL,G22USSRMOR,G22USSDHOL,G22USSLGRA,G22GOVDKEL,G22GOVRSCH,G22GOVIPYL,G22GOVLCOR,G22SOSRSCH,G22SOSDREP,G22SOSLLAN,G22ATGRKOB,G22ATGDMAN,G22TRERJOH,G22TREDROG,G22TRELROB,G22INSRSCH,G22INSDCOR,G22SSC1YES,G22SSC1NO,G22SSC2YES,G22SSC2NO,G22SSC3YES,G22SSC3NO,G22SSC5YES,G22SSC5NO,G22SSC6YES,G22SSC6NO,G22SSC7YES,G22SSC7NO,SL001DROG,SL001RHOU,SL002RCOL,SL003RSMI,SL004RJAC,SL005OWRI,SL005RBAR,SL006DFRI,SL006RPAR,SL007RGOD,SL008DSHE,SL008OWRI,SL008RCRO,SL009DCLO,SL009RGAR,SL010DHAS,SL011DWIL,SL011RBRY,SL012RBLE,SL013RNEW,SL014DMIL,SL014OWRI,SL014RESA,SL015DHOU,SL015OWRI,SL015RBIN,SL016DFEA,SL016OWRI,SL016RROI,SL017DHOY,SL017LKER,SL017OWRI,SL017RCAR,SL018DNEI,SL018OWRI,SL018RGOR,SL019DCLA,SL019OWRI,SL019RRED,SL020DPOS,SL020OWRI,SL020RRAH,SL021DSTO,SL021OWRI,SL022DVAU,SL022OWRI,SL022RCOL,SL023DRUI,SL023OWRI,SL024DOUS,SL024OWRI,SL025DXU,SL025OWRI,SL026DTIF,SL026OWRI,SL026RTHO,SL027DPRI,SL027OWRI,SL027RTAR,SL028DALL,SL028OWRI,SL028RTUR,SL029DMEY,SL029OWRI,SL029RSOF,SL030DEIT,SL030OWRI,SL030RWIL,SL031DRUI,SL031OWRI,SL031RGRI,SL032DCUR,SL032OWRI,SL033DHUT,SL033LBAR,SL033OWRI,SL033RTHO,SL034DWIN,SL034OWRI,SL034RCAB,SL035DII,SL035OWRI,SL035RSTI,SL036DMEL,SL036OWRI,SL036RBRA,SL037DORO,SL037OWRI,SL037RWHI,SL038OWRI,SL038RJOH,SL039DWES,SL039OWRI,SL039RDON,SL040DALL,SL040RBUE,SL041DSCH,SL041RPRO,SL042RNEE,SL043DDAV,SL043OWRI,SL043RSUT,SL044DBAL,SL045DAMY,SL046DHIG,SL047RELL,SL048DOSM,SL048OWRI,SL048RFRE,SL049DBOY,SL049OWRI,SL049RCLA,SL050OWRI,SL050RPAT,SL051RTIT,SL052DFLE,SL052OWRI,SL052RBOR,SL053DHAS,SL053OWRI,SL053RWIL,SL054DJOH,SL054OWRI,SL054RCOR,SL055DSCH,SL055OWRI,SL055RSTA,SL056DWEI,SL056OWRI,SL057DALC,SL057OWRI,SL058DMIL,SL058OWRI,SL059DMCC,SL059RSCH,SL060DMCG,SL060RSCH,SL061RAWE,SL062RGAR,SL063REPL,SL064DSME,SL064RBLO,SL065RUND,SL066DCAR,SL067DZIT,SL067RDOD,SL068DII,SL068RBUT,SL069DCRE,SL069RSAN,SL070RHIL,SL071RHOW,SL072RAND,SL073RMAS,SL074LHEI,SL074ROWE,SL075RCAR,SL076DTOR,SL076RSMI,SL077RWIL,SL078DSHI,SL078OWRI,SL078RESS,SL079DTRI,SL079RROT,SL080RRHI,SL081RCAR,SL082DHOB,SL082RHOW,SL083DHEL,SL084DCAR,SL085DBRI,SL085RPEN,SL086DMIL,SL086RLIN,SL087DSTR,SL087REST,SL088DSCH,SL088RPIC,SL089DOHA,SL090RMAU,SL091DCHO,SL091LBAI,SL091RBER,SL092DCAR,SL093RBER,SL094DHOW,SL094RDEL,SL095DSAW,SL095RPAR,SL096RKES,SL097RHOH,SL098RHOW,SL099RHUM,SL100DMCC,SL100RHAW,SL101RSEI,SL102DPRO,SL102RWHI,SL103DMAR,SL103LHER,SL104DSTR,SL104RWAG,SL105DABE,SL105RLAN,SL106RMOS,SL107RCON,SL108DWOO,SL108OWRI,SL109LSOL,SL109RWAY,SL110RRAH,SL111DHAM,SL111RWAS,SL112RBLE,SL113DROT,SL113RFAI,SL114RMUR,SL115RWHI,SL116DBAR,SL116RHOF,SL117DTRI,SL117OWRI,SL117RTUR,SL118RMIN,SL119RGOE,SL120RSMI,SL121DPIN,SL121OWRI,SL121RRES,SL122RCLI,SL123RLEW,SL124RYOU,SL125RFRA,SU038DLAR,SU038RRYC,geometry
0,Allen,001,000010,Carlyle Township Voting District,28.0,165.0,42.0,0.0,235.0,104.0,19.0,3.0,31.0,93.0,0.0,2.0,96.0,26.0,4.0,94.0,32.0,94.0,26.0,6.0,102.0,23.0,71.0,51.0,69.0,53.0,66.0,56.0,67.0,55.0,65.0,57.0,72.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON Z ((-95.42748 37.96487 0.00000, -95.42..."
1,Allen,001,000020,Cottage

In [149]:
column_sums = final_df.sum(numeric_only=True)
column_sums

REG_DEM        843348.0
REG_REP       1272272.0
REG_UNA        828057.0
REG_LIB         37177.0
REG_TOTAL     2980854.0
G22USSRMOR     867605.0
G22USSDHOL     609390.0
G22USSLGRA      44610.0
G22GOVDKEL     795764.0
G22GOVRSCH     687768.0
G22GOVIPYL      28175.0
G22GOVLCOR      16152.0
G22SOSRSCH     837478.0
G22SOSDREP     629586.0
G22SOSLLAN      40163.0
G22ATGRKOB     723828.0
G22ATGDMAN     786835.0
G22TRERJOH     769284.0
G22TREDROG     665388.0
G22TRELROB      71331.0
G22INSRSCH     894561.0
G22INSDCOR     589275.0
G22SSC1YES     903383.0
G22SSC1NO      433133.0
G22SSC2YES     871551.0
G22SSC2NO      450363.0
G22SSC3YES     862865.0
G22SSC3NO      455065.0
G22SSC5YES     878748.0
G22SSC5NO      440597.0
G22SSC6YES     876579.0
G22SSC6NO      440035.0
G22SSC7YES     967972.0
G22SSC7NO      352300.0
SL001DROG        1909.0
SL001RHOU        5778.0
SL002RCOL        6435.0
SL003RSMI        4714.0
SL004RJAC        7069.0
SL005OWRI          43.0
SL005RBAR        3911.0
SL006DFRI       

In [154]:
final_gdf = gpd.GeoDataFrame(final_df, geometry='geometry')

# Define the output file path
output_shapefile_path = "final_output.shp"

# Save the GeoDataFrame as a shapefile
final_gdf.to_file(output_shapefile_path, driver='ESRI Shapefile')

print(f"Shapefile saved to {output_shapefile_path}")


Shapefile saved to final_output.shp


### Previous code(not using)

Helper Functions

In [150]:
# Function to match precincts in pber and in election results
def compare_vtds(election_results, pber):
    unique_vtd_election_results = set(election_results['VTD'].unique())
    unique_vtd_pber = set(pber['VTDST'].unique())

    matching_vtds = unique_vtd_election_results & unique_vtd_pber
    non_matching_in_election_results = unique_vtd_election_results - unique_vtd_pber
    non_matching_in_pber = unique_vtd_pber - unique_vtd_election_results

    max_length = max(len(matching_vtds), len(non_matching_in_election_results), len(non_matching_in_pber))

    matching_vtds_padded = list(matching_vtds) + [''] * (max_length - len(matching_vtds))
    non_matching_in_election_results_padded = list(non_matching_in_election_results) + [''] * (max_length - len(non_matching_in_election_results))
    non_matching_in_pber_padded = list(non_matching_in_pber) + [''] * (max_length - len(non_matching_in_pber))

    vtd_comparison_df = pd.DataFrame({
        'Matching VTDs': matching_vtds_padded,
        'Not in PBER': non_matching_in_election_results_padded,
        'Not in Election Results': non_matching_in_pber_padded
    })
    
    return vtd_comparison_df

In [151]:
# Function to plot geometries
def plot_precincts(gdf, col_name):
    fig, ax = plt.subplots(figsize=(40, 40))
    gdf.plot(column=col_name, ax=ax, legend=True, cmap='tab20', edgecolor='black', linewidth=0.5)
    for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf[col_name]):
        ax.text(x, y, label, fontsize=15, ha='center')
    plt.show()